In [1]:
import numpy as np
#import os
#import sys
#import time
import pandas as pd 
#from tqdm.notebook import tqdm_notebook
#import pickle
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout
from keras.layers import LSTM
#from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, CSVLogger
#from keras import optimizers
# from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
#import logging
# import talos as ta
from matplotlib import pyplot as plt

In [2]:
#ENTER THE UAH DRIVESET LOCATION
path = 'C:/Users/Geraldo Wibowo/Desktop/Year 4/FYP/JUPYTER TEST 1/UAH-DRIVESET-v1/'

In [3]:
driver = 'D1','D2','D3','D4','D5','D6'
condition = 'NORMAL1_SECONDARY','NORMAL2_SECONDARY','NORMAL_MOTORWAY','AGGRESSIVE_MOTORWAY','DROWSY_MOTORWAY','AGGRESSIVE_SECONDARY','DROWSY_SECONDARY'
parameter = 'RAW_GPS','RAW_ACCELEROMETERS','PROC_LANE_DETECTION','PROL_VEHICLE_DETECTION','PROC_OPENSTREETMAP_DATA','SEMANTIC ONLINE'

In [4]:
#FOLDER PATH
D1_NORMAL1_SECONDARY = 'D1/20151110175712-16km-D1-NORMAL1-SECONDARY'
D1_NORMAL2_SECONDARY = 'D1/20151110180824-16km-D1-NORMAL2-SECONDARY'
D1_NORMAL_MOTORWAY = 'D1/20151111123124-25km-D1-NORMAL-MOTORWAY'
D1_AGGRESSIVE_MOTORWAY = 'D1/20151111125233-24km-D1-AGGRESSIVE-MOTORWAY'
D1_DROWSY_MOTORWAY = 'D1/20151111132348-25km-D1-DROWSY-MOTORWAY'
D1_AGGRESSIVE_SECONDARY = 'D1/20151111134545-16km-D1-AGGRESSIVE-SECONDARY'
D1_DROWSY_SECONDARY = 'D1/20151111135612-13km-D1-DROWSY-SECONDARY'
D2_NORMAL1_SECONDARY = 'D2/20151120160904-16km-D2-NORMAL1-SECONDARY'
D2_NORMAL2_SECONDARY = 'D2/20151120162105-17km-D2-NORMAL2-SECONDARY'
D2_NORMAL_MOTORWAY = 'D2/20151120131714-26km-D2-NORMAL-MOTORWAY'
D2_AGGRESSIVE_MOTORWAY = 'D2/20151120133502-26km-D2-AGGRESSIVE-MOTORWAY'
D2_DROWSY_MOTORWAY = 'D2/20151120135152-25km-D2-DROWSY-MOTORWAY'
D2_AGGRESSIVE_SECONDARY = 'D2/20151120163350-16km-D2-AGGRESSIVE-SECONDARY'
D2_DROWSY_SECONDARY = 'D2/20151120164606-16km-D2-DROWSY-SECONDARY'
D3_NORMAL1_SECONDARY = 'D3/20151126124208-16km-D3-NORMAL1-SECONDARY'
D3_NORMAL2_SECONDARY = 'D3/20151126125458-16km-D3-NORMAL2-SECONDARY'
D3_NORMAL_MOTORWAY = 'D3/20151126110502-26km-D3-NORMAL-MOTORWAY'
D3_AGGRESSIVE_MOTORWAY = 'D3/20151126134736-26km-D3-AGGRESSIVE-MOTORWAY'
D3_DROWSY_MOTORWAY = 'D3/20151126113754-26km-D3-DROWSY-MOTORWAY'
D3_AGGRESSIVE_SECONDARY = 'D3/20151126130707-16km-D3-AGGRESSIVE-SECONDARY'
D3_DROWSY_SECONDARY = 'D3/20151126132013-17km-D3-DROWSY-SECONDARY'
D4_NORMAL1_SECONDARY = 'D4/20151203171800-16km-D4-NORMAL1-SECONDARY'
D4_NORMAL2_SECONDARY = 'D4/20151203173103-17km-D4-NORMAL2-SECONDARY'
D4_NORMAL_MOTORWAY = 'D4/20151204152848-25km-D4-NORMAL-MOTORWAY'
D4_AGGRESSIVE_MOTORWAY = 'D4/20151204154908-25km-D4-AGGRESSIVE-MOTORWAY'
D4_DROWSY_MOTORWAY = 'D4/20151204160823-25km-D4-DROWSY-MOTORWAY'
D4_AGGRESSIVE_SECONDARY = 'D4/20151203174324-16km-D4-AGGRESSIVE-SECONDARY'
D4_DROWSY_SECONDARY = 'D4/20151203175637-17km-D4-DROWSY-SECONDARY'
D5_NORMAL1_SECONDARY = 'D5/20151211162829-16km-D5-NORMAL1-SECONDARY'
D5_NORMAL2_SECONDARY = 'D5/20151211164124-17km-D5-NORMAL2-SECONDARY'
D5_NORMAL_MOTORWAY = 'D5/20151209151242-25km-D5-NORMAL-MOTORWAY'
D5_AGGRESSIVE_MOTORWAY = 'D5/20151209153137-25km-D5-AGGRESSIVE-MOTORWAY'
D5_DROWSY_MOTORWAY = 'D5/20151211160213-25km-D5-DROWSY-MOTORWAY'
D5_AGGRESSIVE_SECONDARY = 'D5/20151211165606-12km-D5-AGGRESSIVE-SECONDARY'
D5_DROWSY_SECONDARY = 'D5/20151211170502-16km-D5-DROWSY-SECONDARY'
D6_NORMAL_SECONDARY = 'D6/20151221112434-17km-D6-NORMAL-SECONDARY'
D6_NORMAL_MOTORWAY = 'D6/20151217162714-26km-D6-NORMAL-MOTORWAY'
D6_AGGRESSIVE_MOTORWAY = 'D6/20151221120051-26km-D6-AGGRESSIVE-MOTORWAY'
D6_DROWSY_MOTORWAY = 'D6/20151217164730-25km-D6-DROWSY-MOTORWAY'
D6_DROWSY_SECONDARY = 'D6/20151221113846-16km-D6-DROWSY-SECONDARY'

In [26]:
def RAW_GPS(path, driver_condition):
    
    driver_condition = path+driver_condition

    RAW_GPS = pd.read_csv (driver_condition+'/RAW_GPS.txt', sep=" ").iloc[:,0:-4]
    RAW_GPS.columns = [
        'Timestamp (seconds)', 
        'Speed (km/h)', 
        'Latitude coordinate (degrees)', 
        'Longitude coordinate (degrees)', 
        'Altitude (meters)', 
        'Vertical accuracy (degrees)', 
        'Horizontal accuracy (degrees)', 
        'Course (degrees)', 
        'Difcourse: course variation (degrees)'
    ] 
    
    return RAW_GPS

def RAW_ACCELEROMETERS(path, driver_condition):
    
    driver_condition = path+driver_condition

    RAW_ACCELEROMETERS = pd.read_csv (driver_condition+'/RAW_ACCELEROMETERS.txt', sep=" ").iloc[:,0:-1]
    RAW_ACCELEROMETERS.columns = [
        'Timestamp (seconds)',
        'Boolean of system activated (1 if >50km/h)',
        'Acceleration in X (Gs)',
        'Acceleration in Y (Gs)',
        'Acceleration in Z (Gs)',
        'Acceleration in X filtered by KF (Gs)',
        'Acceleration in Y filtered by KF (Gs)',
        'Acceleration in Z filtered by KF (Gs)',
        'Roll (degrees)',
        'Pitch (degrees)',
        'Yaw (degrees)']
    return RAW_ACCELEROMETERS

def PROC_LANE_DETECTION(path, driver_condition):
    
    driver_condition = path+driver_condition

    PROC_LANE_DETECTION = pd.read_csv (driver_condition+'/PROC_LANE_DETECTION.txt', sep=" ")
    PROC_LANE_DETECTION.columns = [
        'Timestamp (seconds)',
        'X: car position relative to lane center (meters)',
        'Phi: car angle relative to lane curvature (degrees)',
        'W: road width (meters)',
        "State of the lane det. algorithm [-1=calibrating, 0=initializing, 1=undetected, 2=detected/running]" ]
    return PROC_LANE_DETECTION

def PROC_VEHICLE_DETECTION(path, driver_condition):
    
    driver_condition = path+driver_condition

    PROC_VEHICLE_DETECTION = pd.read_csv (driver_condition+'/PROC_VEHICLE_DETECTION.txt', sep=" ").iloc[:,0:-1]
    PROC_VEHICLE_DETECTION.columns = [
        'Timestamp (seconds)',
        "Distance to ahead vehicle in current lane (meters) [value -1 means no car is detected in front]",
        "Time of impact to ahead vehicle (seconds) [distance related to own speed]",
        "Number of detected vehicles in this frame (traffic)",
        "GPS speed (km/h) [same as in RAW GPS]" ]    
    return PROC_VEHICLE_DETECTION

def PROC_OPENSTREETMAP_DATA(path, driver_condition):
    
    driver_condition = path+driver_condition

    PROC_OPENSTREETMAP_DATA = pd.read_csv (driver_condition+'/PROC_OPENSTREETMAP_DATA.txt', sep=" ")
    PROC_OPENSTREETMAP_DATA.columns = [
        'Timestamp (seconds)',
        "Maximum allowed speed of current road (km/h)",
        "Reliability of obtained maxspeed (0=unknown, 1=reliable, 2=used previously obtained maxspeed,3=estimated by type of road)",
        "Type of road (motorway, trunk, secondary...)",
        "Number of lanes in current road",
        "Estimated current lane (1=right lane, 2=first left lane, 3=second left lane, etc) [experimental]",
        "GPS Latitude used to query OSM (degrees)",
        "GPS Longitude used to query OSM (degrees)",
        "OSM delay to answer query (seconds)",
        "GPS speed (km/h) [same as in RAW GPS]]",] 
    return PROC_OPENSTREETMAP_DATA

def SEMANTIC_ONLINE(path, driver_condition):
    
    driver_condition = path+driver_condition

    SEMANTIC_ONLINE = pd.read_csv (driver_condition+'/SEMANTIC_ONLINE.txt', sep=" ").iloc[:,-5:-2]
    SEMANTIC_ONLINE2 = pd.read_csv (driver_condition+'/SEMANTIC_ONLINE.txt', sep=' ').iloc[:,0:1]
    SEMANTIC_ONLINE.columns = [
        'Ratio normal (base 1)',
        'Ratio drowsy (base 1)',
        'Ratio aggressive (base 1)']
    SEMANTIC_ONLINE.insert(0, 'Timestamp (seconds)', SEMANTIC_ONLINE2, True)
    return SEMANTIC_ONLINE

In [33]:
D1_NORMAL1_SECONDARY_RAW_GPS = RAW_GPS(path, D1_NORMAL1_SECONDARY)
D1_NORMAL1_SECONDARY_RAW_ACCELEROMETERS = RAW_ACCELEROMETERS(path, D1_NORMAL1_SECONDARY)
D1_NORMAL1_SECONDARY_merged = pd.merge_asof(D1_NORMAL1_SECONDARY_RAW_GPS, D1_NORMAL1_SECONDARY_RAW_ACCELEROMETERS, on='Timestamp (seconds)', direction="nearest")
D1_NORMAL1_SECONDARY_PROC_LANE_DETECTION = PROC_LANE_DETECTION(path, D1_NORMAL1_SECONDARY)
D1_NORMAL1_SECONDARY_merged = pd.merge_asof(D1_NORMAL1_SECONDARY_merged, D1_NORMAL1_SECONDARY_PROC_LANE_DETECTION, on='Timestamp (seconds)', direction="nearest")
D1_NORMAL1_SECONDARY_PROC_VEHICLE_DETECTION = PROC_VEHICLE_DETECTION(path, D1_NORMAL1_SECONDARY)
D1_NORMAL1_SECONDARY_merged = pd.merge_asof(D1_NORMAL1_SECONDARY_merged, D1_NORMAL1_SECONDARY_PROC_VEHICLE_DETECTION, on='Timestamp (seconds)', direction="nearest")
D1_NORMAL1_SECONDARY_PROC_OPENSTREETMAP_DATA = PROC_OPENSTREETMAP_DATA(path, D1_NORMAL1_SECONDARY)
D1_NORMAL1_SECONDARY_merged = pd.merge_asof(D1_NORMAL1_SECONDARY_merged, D1_NORMAL1_SECONDARY_PROC_OPENSTREETMAP_DATA, on='Timestamp (seconds)', direction="nearest")
D1_NORMAL1_SECONDARY_SEMANTIC_ONLINE = SEMANTIC_ONLINE(path, D1_NORMAL1_SECONDARY)
D1_NORMAL1_SECONDARY_merged = pd.merge_asof(D1_NORMAL1_SECONDARY_merged, D1_NORMAL1_SECONDARY_SEMANTIC_ONLINE, on='Timestamp (seconds)', direction="nearest")


D1_NORMAL2_SECONDARY_RAW_GPS = RAW_GPS(path, D1_NORMAL2_SECONDARY)
D1_NORMAL2_SECONDARY_RAW_ACCELEROMETERS = RAW_ACCELEROMETERS(path, D1_NORMAL2_SECONDARY)
D1_NORMAL2_SECONDARY_merged = pd.merge_asof(D1_NORMAL2_SECONDARY_RAW_GPS, D1_NORMAL2_SECONDARY_RAW_ACCELEROMETERS, on='Timestamp (seconds)', direction="nearest")
D1_NORMAL2_SECONDARY_PROC_LANE_DETECTION = PROC_LANE_DETECTION(path, D1_NORMAL2_SECONDARY)
D1_NORMAL2_SECONDARY_merged = pd.merge_asof(D1_NORMAL2_SECONDARY_merged, D1_NORMAL2_SECONDARY_PROC_LANE_DETECTION, on='Timestamp (seconds)', direction="nearest")
D1_NORMAL2_SECONDARY_PROC_VEHICLE_DETECTION = PROC_VEHICLE_DETECTION(path, D1_NORMAL2_SECONDARY)
D1_NORMAL2_SECONDARY_merged = pd.merge_asof(D1_NORMAL2_SECONDARY_merged, D1_NORMAL2_SECONDARY_PROC_VEHICLE_DETECTION, on='Timestamp (seconds)', direction="nearest")
D1_NORMAL2_SECONDARY_PROC_OPENSTREETMAP_DATA = PROC_OPENSTREETMAP_DATA(path, D1_NORMAL2_SECONDARY)
D1_NORMAL2_SECONDARY_merged = pd.merge_asof(D1_NORMAL2_SECONDARY_merged, D1_NORMAL2_SECONDARY_PROC_OPENSTREETMAP_DATA, on='Timestamp (seconds)', direction="nearest")
D1_NORMAL2_SECONDARY_SEMANTIC_ONLINE = SEMANTIC_ONLINE(path, D1_NORMAL2_SECONDARY)
D1_NORMAL2_SECONDARY_merged = pd.merge_asof(D1_NORMAL2_SECONDARY_merged, D1_NORMAL2_SECONDARY_SEMANTIC_ONLINE, on='Timestamp (seconds)', direction="nearest")


D1_NORMAL_MOTORWAY_RAW_GPS = RAW_GPS(path, D1_NORMAL_MOTORWAY)
D1_NORMAL_MOTORWAY_RAW_ACCELEROMETERS = RAW_ACCELEROMETERS(path, D1_NORMAL_MOTORWAY)
D1_NORMAL_MOTORWAY_merged = pd.merge_asof(D1_NORMAL_MOTORWAY_RAW_GPS, D1_NORMAL_MOTORWAY_RAW_ACCELEROMETERS, on='Timestamp (seconds)', direction="nearest")
D1_NORMAL_MOTORWAY_PROC_LANE_DETECTION = PROC_LANE_DETECTION(path, D1_NORMAL_MOTORWAY)
D1_NORMAL_MOTORWAY_merged = pd.merge_asof(D1_NORMAL_MOTORWAY_merged, D1_NORMAL_MOTORWAY_PROC_LANE_DETECTION, on='Timestamp (seconds)', direction="nearest")
D1_NORMAL_MOTORWAY_PROC_VEHICLE_DETECTION = PROC_VEHICLE_DETECTION(path, D1_NORMAL_MOTORWAY)
D1_NORMAL_MOTORWAY_merged = pd.merge_asof(D1_NORMAL_MOTORWAY_merged, D1_NORMAL_MOTORWAY_PROC_VEHICLE_DETECTION, on='Timestamp (seconds)', direction="nearest")
D1_NORMAL_MOTORWAY_PROC_OPENSTREETMAP_DATA = PROC_OPENSTREETMAP_DATA(path, D1_NORMAL_MOTORWAY)
D1_NORMAL_MOTORWAY_merged = pd.merge_asof(D1_NORMAL_MOTORWAY_merged, D1_NORMAL_MOTORWAY_PROC_OPENSTREETMAP_DATA, on='Timestamp (seconds)', direction="nearest")
D1_NORMAL_MOTORWAY_SEMANTIC_ONLINE = SEMANTIC_ONLINE(path, D1_NORMAL_MOTORWAY)
D1_NORMAL_MOTORWAY_merged = pd.merge_asof(D1_NORMAL_MOTORWAY_merged, D1_NORMAL_MOTORWAY_SEMANTIC_ONLINE, on='Timestamp (seconds)', direction="nearest")


D1_AGGRESSIVE_MOTORWAY_RAW_GPS = RAW_GPS(path, D1_AGGRESSIVE_MOTORWAY)
D1_AGGRESSIVE_MOTORWAY_RAW_ACCELEROMETERS = RAW_ACCELEROMETERS(path, D1_AGGRESSIVE_MOTORWAY)
D1_AGGRESSIVE_MOTORWAY_merged = pd.merge_asof(D1_AGGRESSIVE_MOTORWAY_RAW_GPS, D1_AGGRESSIVE_MOTORWAY_RAW_ACCELEROMETERS, on='Timestamp (seconds)', direction="nearest")
D1_AGGRESSIVE_MOTORWAY_PROC_LANE_DETECTION = PROC_LANE_DETECTION(path, D1_AGGRESSIVE_MOTORWAY)
D1_AGGRESSIVE_MOTORWAY_merged = pd.merge_asof(D1_AGGRESSIVE_MOTORWAY_merged, D1_AGGRESSIVE_MOTORWAY_PROC_LANE_DETECTION, on='Timestamp (seconds)', direction="nearest")
D1_AGGRESSIVE_MOTORWAY_PROC_VEHICLE_DETECTION = PROC_VEHICLE_DETECTION(path, D1_AGGRESSIVE_MOTORWAY)
D1_AGGRESSIVE_MOTORWAY_merged = pd.merge_asof(D1_AGGRESSIVE_MOTORWAY_merged, D1_AGGRESSIVE_MOTORWAY_PROC_VEHICLE_DETECTION, on='Timestamp (seconds)', direction="nearest")
D1_AGGRESSIVE_MOTORWAY_PROC_OPENSTREETMAP_DATA = PROC_OPENSTREETMAP_DATA(path, D1_AGGRESSIVE_MOTORWAY)
D1_AGGRESSIVE_MOTORWAY_merged = pd.merge_asof(D1_AGGRESSIVE_MOTORWAY_merged, D1_AGGRESSIVE_MOTORWAY_PROC_OPENSTREETMAP_DATA, on='Timestamp (seconds)', direction="nearest")
D1_AGGRESSIVE_MOTORWAY_SEMANTIC_ONLINE = SEMANTIC_ONLINE(path, D1_AGGRESSIVE_MOTORWAY)
D1_AGGRESSIVE_MOTORWAY_merged = pd.merge_asof(D1_AGGRESSIVE_MOTORWAY_merged, D1_AGGRESSIVE_MOTORWAY_SEMANTIC_ONLINE, on='Timestamp (seconds)', direction="nearest")


D1_DROWSY_MOTORWAY_RAW_GPS = RAW_GPS(path, D1_DROWSY_MOTORWAY)
D1_DROWSY_MOTORWAY_RAW_ACCELEROMETERS = RAW_ACCELEROMETERS(path, D1_DROWSY_MOTORWAY)
D1_DROWSY_MOTORWAY_merged = pd.merge_asof(D1_DROWSY_MOTORWAY_RAW_GPS, D1_DROWSY_MOTORWAY_RAW_ACCELEROMETERS, on='Timestamp (seconds)', direction="nearest")
D1_DROWSY_MOTORWAY_PROC_LANE_DETECTION = PROC_LANE_DETECTION(path, D1_DROWSY_MOTORWAY)
D1_DROWSY_MOTORWAY_merged = pd.merge_asof(D1_DROWSY_MOTORWAY_merged, D1_DROWSY_MOTORWAY_PROC_LANE_DETECTION, on='Timestamp (seconds)', direction="nearest")
D1_DROWSY_MOTORWAY_PROC_VEHICLE_DETECTION = PROC_VEHICLE_DETECTION(path, D1_DROWSY_MOTORWAY)
D1_DROWSY_MOTORWAY_merged = pd.merge_asof(D1_DROWSY_MOTORWAY_merged, D1_DROWSY_MOTORWAY_PROC_VEHICLE_DETECTION, on='Timestamp (seconds)', direction="nearest")
D1_DROWSY_MOTORWAY_PROC_OPENSTREETMAP_DATA = PROC_OPENSTREETMAP_DATA(path, D1_DROWSY_MOTORWAY)
D1_DROWSY_MOTORWAY_merged = pd.merge_asof(D1_DROWSY_MOTORWAY_merged, D1_DROWSY_MOTORWAY_PROC_OPENSTREETMAP_DATA, on='Timestamp (seconds)', direction="nearest")
D1_DROWSY_MOTORWAY_SEMANTIC_ONLINE = SEMANTIC_ONLINE(path, D1_DROWSY_MOTORWAY)
D1_DROWSY_MOTORWAY_merged = pd.merge_asof(D1_DROWSY_MOTORWAY_merged, D1_DROWSY_MOTORWAY_SEMANTIC_ONLINE, on='Timestamp (seconds)', direction="nearest")


D1_AGGRESSIVE_SECONDARY_RAW_GPS = RAW_GPS(path, D1_AGGRESSIVE_SECONDARY)
D1_AGGRESSIVE_SECONDARY_RAW_ACCELEROMETERS = RAW_ACCELEROMETERS(path, D1_AGGRESSIVE_SECONDARY)
D1_AGGRESSIVE_SECONDARY_merged = pd.merge_asof(D1_AGGRESSIVE_SECONDARY_RAW_GPS, D1_AGGRESSIVE_SECONDARY_RAW_ACCELEROMETERS, on='Timestamp (seconds)', direction="nearest")
D1_AGGRESSIVE_SECONDARY_PROC_LANE_DETECTION = PROC_LANE_DETECTION(path, D1_AGGRESSIVE_SECONDARY)
D1_AGGRESSIVE_SECONDARY_merged = pd.merge_asof(D1_AGGRESSIVE_SECONDARY_merged, D1_AGGRESSIVE_SECONDARY_PROC_LANE_DETECTION, on='Timestamp (seconds)', direction="nearest")
D1_AGGRESSIVE_SECONDARY_PROC_VEHICLE_DETECTION = PROC_VEHICLE_DETECTION(path, D1_AGGRESSIVE_SECONDARY)
D1_AGGRESSIVE_SECONDARY_merged = pd.merge_asof(D1_AGGRESSIVE_SECONDARY_merged, D1_AGGRESSIVE_SECONDARY_PROC_VEHICLE_DETECTION, on='Timestamp (seconds)', direction="nearest")
D1_AGGRESSIVE_SECONDARY_PROC_OPENSTREETMAP_DATA = PROC_OPENSTREETMAP_DATA(path, D1_AGGRESSIVE_SECONDARY)
D1_AGGRESSIVE_SECONDARY_merged = pd.merge_asof(D1_AGGRESSIVE_SECONDARY_merged, D1_AGGRESSIVE_SECONDARY_PROC_OPENSTREETMAP_DATA, on='Timestamp (seconds)', direction="nearest")
D1_AGGRESSIVE_SECONDARY_SEMANTIC_ONLINE = SEMANTIC_ONLINE(path, D1_AGGRESSIVE_SECONDARY)
D1_AGGRESSIVE_SECONDARY_merged = pd.merge_asof(D1_AGGRESSIVE_SECONDARY_merged, D1_AGGRESSIVE_SECONDARY_SEMANTIC_ONLINE, on='Timestamp (seconds)', direction="nearest")


D1_DROWSY_SECONDARY_RAW_GPS = RAW_GPS(path, D1_DROWSY_SECONDARY)
D1_DROWSY_SECONDARY_RAW_ACCELEROMETERS = RAW_ACCELEROMETERS(path, D1_DROWSY_SECONDARY)
D1_DROWSY_SECONDARY_merged = pd.merge_asof(D1_DROWSY_SECONDARY_RAW_GPS, D1_DROWSY_SECONDARY_RAW_ACCELEROMETERS, on='Timestamp (seconds)', direction="nearest")
D1_DROWSY_SECONDARY_PROC_LANE_DETECTION = PROC_LANE_DETECTION(path, D1_DROWSY_SECONDARY)
D1_DROWSY_SECONDARY_merged = pd.merge_asof(D1_DROWSY_SECONDARY_merged, D1_DROWSY_SECONDARY_PROC_LANE_DETECTION, on='Timestamp (seconds)', direction="nearest")
D1_DROWSY_SECONDARY_PROC_VEHICLE_DETECTION = PROC_VEHICLE_DETECTION(path, D1_DROWSY_SECONDARY)
D1_DROWSY_SECONDARY_merged = pd.merge_asof(D1_DROWSY_SECONDARY_merged, D1_DROWSY_SECONDARY_PROC_VEHICLE_DETECTION, on='Timestamp (seconds)', direction="nearest")
D1_DROWSY_SECONDARY_PROC_OPENSTREETMAP_DATA = PROC_OPENSTREETMAP_DATA(path, D1_DROWSY_SECONDARY)
D1_DROWSY_SECONDARY_merged = pd.merge_asof(D1_DROWSY_SECONDARY_merged, D1_DROWSY_SECONDARY_PROC_OPENSTREETMAP_DATA, on='Timestamp (seconds)', direction="nearest")
D1_DROWSY_SECONDARY_SEMANTIC_ONLINE = SEMANTIC_ONLINE(path, D1_DROWSY_SECONDARY)
D1_DROWSY_SECONDARY_merged = pd.merge_asof(D1_DROWSY_SECONDARY_merged, D1_DROWSY_SECONDARY_SEMANTIC_ONLINE, on='Timestamp (seconds)', direction="nearest")


In [46]:
def PROC_OPENSTREETMAP_DATA(path, driver_condition):
    
    driver_condition = path+driver_condition

    PROC_OPENSTREETMAP_DATA = pd.read_csv (driver_condition+'/PROC_OPENSTREETMAP_DATA.txt', sep=" ").iloc[:,0:-1]
    PROC_OPENSTREETMAP_DATA.columns = [
        'Timestamp (seconds)',
        "Maximum allowed speed of current road (km/h)",
        "Reliability of obtained maxspeed (0=unknown, 1=reliable, 2=used previously obtained maxspeed,3=estimated by type of road)",
        "Type of road (motorway, trunk, secondary...)",
        "Number of lanes in current road",
        "Estimated current lane (1=right lane, 2=first left lane, 3=second left lane, etc) [experimental]",
        "GPS Latitude used to query OSM (degrees)",
        "GPS Longitude used to query OSM (degrees)",
        "OSM delay to answer query (seconds)",
        "GPS speed (km/h) [same as in RAW GPS]]"] 
    return PROC_OPENSTREETMAP_DATA


In [47]:
D2_NORMAL1_SECONDARY_RAW_GPS = RAW_GPS(path, D2_NORMAL1_SECONDARY)
D2_NORMAL1_SECONDARY_RAW_ACCELEROMETERS = RAW_ACCELEROMETERS(path, D2_NORMAL1_SECONDARY)
D2_NORMAL1_SECONDARY_merged = pd.merge_asof(D2_NORMAL1_SECONDARY_RAW_GPS, D2_NORMAL1_SECONDARY_RAW_ACCELEROMETERS, on='Timestamp (seconds)', direction="nearest")
D2_NORMAL1_SECONDARY_PROC_LANE_DETECTION = PROC_LANE_DETECTION(path, D2_NORMAL1_SECONDARY)
D2_NORMAL1_SECONDARY_merged = pd.merge_asof(D2_NORMAL1_SECONDARY_merged, D2_NORMAL1_SECONDARY_PROC_LANE_DETECTION, on='Timestamp (seconds)', direction="nearest")
D2_NORMAL1_SECONDARY_PROC_VEHICLE_DETECTION = PROC_VEHICLE_DETECTION(path, D2_NORMAL1_SECONDARY)
D2_NORMAL1_SECONDARY_merged = pd.merge_asof(D2_NORMAL1_SECONDARY_merged, D2_NORMAL1_SECONDARY_PROC_VEHICLE_DETECTION, on='Timestamp (seconds)', direction="nearest")
D2_NORMAL1_SECONDARY_PROC_OPENSTREETMAP_DATA = PROC_OPENSTREETMAP_DATA(path, D2_NORMAL1_SECONDARY)
D2_NORMAL1_SECONDARY_merged = pd.merge_asof(D2_NORMAL1_SECONDARY_merged, D2_NORMAL1_SECONDARY_PROC_OPENSTREETMAP_DATA, on='Timestamp (seconds)', direction="nearest")
D2_NORMAL1_SECONDARY_SEMANTIC_ONLINE = SEMANTIC_ONLINE(path, D2_NORMAL1_SECONDARY)
D2_NORMAL1_SECONDARY_merged = pd.merge_asof(D2_NORMAL1_SECONDARY_merged, D2_NORMAL1_SECONDARY_SEMANTIC_ONLINE, on='Timestamp (seconds)', direction="nearest")


D2_NORMAL2_SECONDARY_RAW_GPS = RAW_GPS(path, D2_NORMAL2_SECONDARY)
D2_NORMAL2_SECONDARY_RAW_ACCELEROMETERS = RAW_ACCELEROMETERS(path, D2_NORMAL2_SECONDARY)
D2_NORMAL2_SECONDARY_merged = pd.merge_asof(D2_NORMAL2_SECONDARY_RAW_GPS, D2_NORMAL2_SECONDARY_RAW_ACCELEROMETERS, on='Timestamp (seconds)', direction="nearest")
D2_NORMAL2_SECONDARY_PROC_LANE_DETECTION = PROC_LANE_DETECTION(path, D2_NORMAL2_SECONDARY)
D2_NORMAL2_SECONDARY_merged = pd.merge_asof(D2_NORMAL2_SECONDARY_merged, D2_NORMAL2_SECONDARY_PROC_LANE_DETECTION, on='Timestamp (seconds)', direction="nearest")
D2_NORMAL2_SECONDARY_PROC_VEHICLE_DETECTION = PROC_VEHICLE_DETECTION(path, D2_NORMAL2_SECONDARY)
D2_NORMAL2_SECONDARY_merged = pd.merge_asof(D2_NORMAL2_SECONDARY_merged, D2_NORMAL2_SECONDARY_PROC_VEHICLE_DETECTION, on='Timestamp (seconds)', direction="nearest")
D2_NORMAL2_SECONDARY_PROC_OPENSTREETMAP_DATA = PROC_OPENSTREETMAP_DATA(path, D2_NORMAL2_SECONDARY)
D2_NORMAL2_SECONDARY_merged = pd.merge_asof(D2_NORMAL2_SECONDARY_merged, D2_NORMAL2_SECONDARY_PROC_OPENSTREETMAP_DATA, on='Timestamp (seconds)', direction="nearest")
D2_NORMAL2_SECONDARY_SEMANTIC_ONLINE = SEMANTIC_ONLINE(path, D2_NORMAL2_SECONDARY)
D2_NORMAL2_SECONDARY_merged = pd.merge_asof(D2_NORMAL2_SECONDARY_merged, D2_NORMAL2_SECONDARY_SEMANTIC_ONLINE, on='Timestamp (seconds)', direction="nearest")


D2_NORMAL_MOTORWAY_RAW_GPS = RAW_GPS(path, D2_NORMAL_MOTORWAY)
D2_NORMAL_MOTORWAY_RAW_ACCELEROMETERS = RAW_ACCELEROMETERS(path, D2_NORMAL_MOTORWAY)
D2_NORMAL_MOTORWAY_merged = pd.merge_asof(D2_NORMAL_MOTORWAY_RAW_GPS, D2_NORMAL_MOTORWAY_RAW_ACCELEROMETERS, on='Timestamp (seconds)', direction="nearest")
D2_NORMAL_MOTORWAY_PROC_LANE_DETECTION = PROC_LANE_DETECTION(path, D2_NORMAL_MOTORWAY)
D2_NORMAL_MOTORWAY_merged = pd.merge_asof(D2_NORMAL_MOTORWAY_merged, D2_NORMAL_MOTORWAY_PROC_LANE_DETECTION, on='Timestamp (seconds)', direction="nearest")
D2_NORMAL_MOTORWAY_PROC_VEHICLE_DETECTION = PROC_VEHICLE_DETECTION(path, D2_NORMAL_MOTORWAY)
D2_NORMAL_MOTORWAY_merged = pd.merge_asof(D2_NORMAL_MOTORWAY_merged, D2_NORMAL_MOTORWAY_PROC_VEHICLE_DETECTION, on='Timestamp (seconds)', direction="nearest")
D2_NORMAL_MOTORWAY_PROC_OPENSTREETMAP_DATA = PROC_OPENSTREETMAP_DATA(path, D2_NORMAL_MOTORWAY)
D2_NORMAL_MOTORWAY_merged = pd.merge_asof(D2_NORMAL_MOTORWAY_merged, D2_NORMAL_MOTORWAY_PROC_OPENSTREETMAP_DATA, on='Timestamp (seconds)', direction="nearest")
D2_NORMAL_MOTORWAY_SEMANTIC_ONLINE = SEMANTIC_ONLINE(path, D2_NORMAL_MOTORWAY)
D2_NORMAL_MOTORWAY_merged = pd.merge_asof(D2_NORMAL_MOTORWAY_merged, D2_NORMAL_MOTORWAY_SEMANTIC_ONLINE, on='Timestamp (seconds)', direction="nearest")


D2_AGGRESSIVE_MOTORWAY_RAW_GPS = RAW_GPS(path, D2_AGGRESSIVE_MOTORWAY)
D2_AGGRESSIVE_MOTORWAY_RAW_ACCELEROMETERS = RAW_ACCELEROMETERS(path, D2_AGGRESSIVE_MOTORWAY)
D2_AGGRESSIVE_MOTORWAY_merged = pd.merge_asof(D2_AGGRESSIVE_MOTORWAY_RAW_GPS, D2_AGGRESSIVE_MOTORWAY_RAW_ACCELEROMETERS, on='Timestamp (seconds)', direction="nearest")
D2_AGGRESSIVE_MOTORWAY_PROC_LANE_DETECTION = PROC_LANE_DETECTION(path, D2_AGGRESSIVE_MOTORWAY)
D2_AGGRESSIVE_MOTORWAY_merged = pd.merge_asof(D2_AGGRESSIVE_MOTORWAY_merged, D2_AGGRESSIVE_MOTORWAY_PROC_LANE_DETECTION, on='Timestamp (seconds)', direction="nearest")
D2_AGGRESSIVE_MOTORWAY_PROC_VEHICLE_DETECTION = PROC_VEHICLE_DETECTION(path, D2_AGGRESSIVE_MOTORWAY)
D2_AGGRESSIVE_MOTORWAY_merged = pd.merge_asof(D2_AGGRESSIVE_MOTORWAY_merged, D2_AGGRESSIVE_MOTORWAY_PROC_VEHICLE_DETECTION, on='Timestamp (seconds)', direction="nearest")
D2_AGGRESSIVE_MOTORWAY_PROC_OPENSTREETMAP_DATA = PROC_OPENSTREETMAP_DATA(path, D2_AGGRESSIVE_MOTORWAY)
D2_AGGRESSIVE_MOTORWAY_merged = pd.merge_asof(D2_AGGRESSIVE_MOTORWAY_merged, D2_AGGRESSIVE_MOTORWAY_PROC_OPENSTREETMAP_DATA, on='Timestamp (seconds)', direction="nearest")
D2_AGGRESSIVE_MOTORWAY_SEMANTIC_ONLINE = SEMANTIC_ONLINE(path, D2_AGGRESSIVE_MOTORWAY)
D2_AGGRESSIVE_MOTORWAY_merged = pd.merge_asof(D2_AGGRESSIVE_MOTORWAY_merged, D2_AGGRESSIVE_MOTORWAY_SEMANTIC_ONLINE, on='Timestamp (seconds)', direction="nearest")


D2_DROWSY_MOTORWAY_RAW_GPS = RAW_GPS(path, D2_DROWSY_MOTORWAY)
D2_DROWSY_MOTORWAY_RAW_ACCELEROMETERS = RAW_ACCELEROMETERS(path, D2_DROWSY_MOTORWAY)
D2_DROWSY_MOTORWAY_merged = pd.merge_asof(D2_DROWSY_MOTORWAY_RAW_GPS, D2_DROWSY_MOTORWAY_RAW_ACCELEROMETERS, on='Timestamp (seconds)', direction="nearest")
D2_DROWSY_MOTORWAY_PROC_LANE_DETECTION = PROC_LANE_DETECTION(path, D2_DROWSY_MOTORWAY)
D2_DROWSY_MOTORWAY_merged = pd.merge_asof(D2_DROWSY_MOTORWAY_merged, D2_DROWSY_MOTORWAY_PROC_LANE_DETECTION, on='Timestamp (seconds)', direction="nearest")
D2_DROWSY_MOTORWAY_PROC_VEHICLE_DETECTION = PROC_VEHICLE_DETECTION(path, D2_DROWSY_MOTORWAY)
D2_DROWSY_MOTORWAY_merged = pd.merge_asof(D2_DROWSY_MOTORWAY_merged, D2_DROWSY_MOTORWAY_PROC_VEHICLE_DETECTION, on='Timestamp (seconds)', direction="nearest")
D2_DROWSY_MOTORWAY_PROC_OPENSTREETMAP_DATA = PROC_OPENSTREETMAP_DATA(path, D2_DROWSY_MOTORWAY)
D2_DROWSY_MOTORWAY_merged = pd.merge_asof(D2_DROWSY_MOTORWAY_merged, D2_DROWSY_MOTORWAY_PROC_OPENSTREETMAP_DATA, on='Timestamp (seconds)', direction="nearest")
D2_DROWSY_MOTORWAY_SEMANTIC_ONLINE = SEMANTIC_ONLINE(path, D2_DROWSY_MOTORWAY)
D2_DROWSY_MOTORWAY_merged = pd.merge_asof(D2_DROWSY_MOTORWAY_merged, D2_DROWSY_MOTORWAY_SEMANTIC_ONLINE, on='Timestamp (seconds)', direction="nearest")


D2_AGGRESSIVE_SECONDARY_RAW_GPS = RAW_GPS(path, D2_AGGRESSIVE_SECONDARY)
D2_AGGRESSIVE_SECONDARY_RAW_ACCELEROMETERS = RAW_ACCELEROMETERS(path, D2_AGGRESSIVE_SECONDARY)
D2_AGGRESSIVE_SECONDARY_merged = pd.merge_asof(D2_AGGRESSIVE_SECONDARY_RAW_GPS, D2_AGGRESSIVE_SECONDARY_RAW_ACCELEROMETERS, on='Timestamp (seconds)', direction="nearest")
D2_AGGRESSIVE_SECONDARY_PROC_LANE_DETECTION = PROC_LANE_DETECTION(path, D2_AGGRESSIVE_SECONDARY)
D2_AGGRESSIVE_SECONDARY_merged = pd.merge_asof(D2_AGGRESSIVE_SECONDARY_merged, D2_AGGRESSIVE_SECONDARY_PROC_LANE_DETECTION, on='Timestamp (seconds)', direction="nearest")
D2_AGGRESSIVE_SECONDARY_PROC_VEHICLE_DETECTION = PROC_VEHICLE_DETECTION(path, D2_AGGRESSIVE_SECONDARY)
D2_AGGRESSIVE_SECONDARY_merged = pd.merge_asof(D2_AGGRESSIVE_SECONDARY_merged, D2_AGGRESSIVE_SECONDARY_PROC_VEHICLE_DETECTION, on='Timestamp (seconds)', direction="nearest")
D2_AGGRESSIVE_SECONDARY_PROC_OPENSTREETMAP_DATA = PROC_OPENSTREETMAP_DATA(path, D2_AGGRESSIVE_SECONDARY)
D2_AGGRESSIVE_SECONDARY_merged = pd.merge_asof(D2_AGGRESSIVE_SECONDARY_merged, D2_AGGRESSIVE_SECONDARY_PROC_OPENSTREETMAP_DATA, on='Timestamp (seconds)', direction="nearest")
D2_AGGRESSIVE_SECONDARY_SEMANTIC_ONLINE = SEMANTIC_ONLINE(path, D2_AGGRESSIVE_SECONDARY)
D2_AGGRESSIVE_SECONDARY_merged = pd.merge_asof(D2_AGGRESSIVE_SECONDARY_merged, D2_AGGRESSIVE_SECONDARY_SEMANTIC_ONLINE, on='Timestamp (seconds)', direction="nearest")


D2_DROWSY_SECONDARY_RAW_GPS = RAW_GPS(path, D2_DROWSY_SECONDARY)
D2_DROWSY_SECONDARY_RAW_ACCELEROMETERS = RAW_ACCELEROMETERS(path, D2_DROWSY_SECONDARY)
D2_DROWSY_SECONDARY_merged = pd.merge_asof(D2_DROWSY_SECONDARY_RAW_GPS, D2_DROWSY_SECONDARY_RAW_ACCELEROMETERS, on='Timestamp (seconds)', direction="nearest")
D2_DROWSY_SECONDARY_PROC_LANE_DETECTION = PROC_LANE_DETECTION(path, D2_DROWSY_SECONDARY)
D2_DROWSY_SECONDARY_merged = pd.merge_asof(D2_DROWSY_SECONDARY_merged, D2_DROWSY_SECONDARY_PROC_LANE_DETECTION, on='Timestamp (seconds)', direction="nearest")
D2_DROWSY_SECONDARY_PROC_VEHICLE_DETECTION = PROC_VEHICLE_DETECTION(path, D2_DROWSY_SECONDARY)
D2_DROWSY_SECONDARY_merged = pd.merge_asof(D2_DROWSY_SECONDARY_merged, D2_DROWSY_SECONDARY_PROC_VEHICLE_DETECTION, on='Timestamp (seconds)', direction="nearest")
D2_DROWSY_SECONDARY_PROC_OPENSTREETMAP_DATA = PROC_OPENSTREETMAP_DATA(path, D2_DROWSY_SECONDARY)
D2_DROWSY_SECONDARY_merged = pd.merge_asof(D2_DROWSY_SECONDARY_merged, D2_DROWSY_SECONDARY_PROC_OPENSTREETMAP_DATA, on='Timestamp (seconds)', direction="nearest")
D2_DROWSY_SECONDARY_SEMANTIC_ONLINE = SEMANTIC_ONLINE(path, D2_DROWSY_SECONDARY)
D2_DROWSY_SECONDARY_merged = pd.merge_asof(D2_DROWSY_SECONDARY_merged, D2_DROWSY_SECONDARY_SEMANTIC_ONLINE, on='Timestamp (seconds)', direction="nearest")


In [48]:
D3_NORMAL1_SECONDARY_RAW_GPS = RAW_GPS(path, D3_NORMAL1_SECONDARY)
D3_NORMAL1_SECONDARY_RAW_ACCELEROMETERS = RAW_ACCELEROMETERS(path, D3_NORMAL1_SECONDARY)
D3_NORMAL1_SECONDARY_merged = pd.merge_asof(D3_NORMAL1_SECONDARY_RAW_GPS, D3_NORMAL1_SECONDARY_RAW_ACCELEROMETERS, on='Timestamp (seconds)', direction="nearest")
D3_NORMAL1_SECONDARY_PROC_LANE_DETECTION = PROC_LANE_DETECTION(path, D3_NORMAL1_SECONDARY)
D3_NORMAL1_SECONDARY_merged = pd.merge_asof(D3_NORMAL1_SECONDARY_merged, D3_NORMAL1_SECONDARY_PROC_LANE_DETECTION, on='Timestamp (seconds)', direction="nearest")
D3_NORMAL1_SECONDARY_PROC_VEHICLE_DETECTION = PROC_VEHICLE_DETECTION(path, D3_NORMAL1_SECONDARY)
D3_NORMAL1_SECONDARY_merged = pd.merge_asof(D3_NORMAL1_SECONDARY_merged, D3_NORMAL1_SECONDARY_PROC_VEHICLE_DETECTION, on='Timestamp (seconds)', direction="nearest")
D3_NORMAL1_SECONDARY_PROC_OPENSTREETMAP_DATA = PROC_OPENSTREETMAP_DATA(path, D3_NORMAL1_SECONDARY)
D3_NORMAL1_SECONDARY_merged = pd.merge_asof(D3_NORMAL1_SECONDARY_merged, D3_NORMAL1_SECONDARY_PROC_OPENSTREETMAP_DATA, on='Timestamp (seconds)', direction="nearest")
D3_NORMAL1_SECONDARY_SEMANTIC_ONLINE = SEMANTIC_ONLINE(path, D3_NORMAL1_SECONDARY)
D3_NORMAL1_SECONDARY_merged = pd.merge_asof(D3_NORMAL1_SECONDARY_merged, D3_NORMAL1_SECONDARY_SEMANTIC_ONLINE, on='Timestamp (seconds)', direction="nearest")


D3_NORMAL2_SECONDARY_RAW_GPS = RAW_GPS(path, D3_NORMAL2_SECONDARY)
D3_NORMAL2_SECONDARY_RAW_ACCELEROMETERS = RAW_ACCELEROMETERS(path, D3_NORMAL2_SECONDARY)
D3_NORMAL2_SECONDARY_merged = pd.merge_asof(D3_NORMAL2_SECONDARY_RAW_GPS, D3_NORMAL2_SECONDARY_RAW_ACCELEROMETERS, on='Timestamp (seconds)', direction="nearest")
D3_NORMAL2_SECONDARY_PROC_LANE_DETECTION = PROC_LANE_DETECTION(path, D3_NORMAL2_SECONDARY)
D3_NORMAL2_SECONDARY_merged = pd.merge_asof(D3_NORMAL2_SECONDARY_merged, D3_NORMAL2_SECONDARY_PROC_LANE_DETECTION, on='Timestamp (seconds)', direction="nearest")
D3_NORMAL2_SECONDARY_PROC_VEHICLE_DETECTION = PROC_VEHICLE_DETECTION(path, D3_NORMAL2_SECONDARY)
D3_NORMAL2_SECONDARY_merged = pd.merge_asof(D3_NORMAL2_SECONDARY_merged, D3_NORMAL2_SECONDARY_PROC_VEHICLE_DETECTION, on='Timestamp (seconds)', direction="nearest")
D3_NORMAL2_SECONDARY_PROC_OPENSTREETMAP_DATA = PROC_OPENSTREETMAP_DATA(path, D3_NORMAL2_SECONDARY)
D3_NORMAL2_SECONDARY_merged = pd.merge_asof(D3_NORMAL2_SECONDARY_merged, D3_NORMAL2_SECONDARY_PROC_OPENSTREETMAP_DATA, on='Timestamp (seconds)', direction="nearest")
D3_NORMAL2_SECONDARY_SEMANTIC_ONLINE = SEMANTIC_ONLINE(path, D3_NORMAL2_SECONDARY)
D3_NORMAL2_SECONDARY_merged = pd.merge_asof(D3_NORMAL2_SECONDARY_merged, D3_NORMAL2_SECONDARY_SEMANTIC_ONLINE, on='Timestamp (seconds)', direction="nearest")


D3_NORMAL_MOTORWAY_RAW_GPS = RAW_GPS(path, D3_NORMAL_MOTORWAY)
D3_NORMAL_MOTORWAY_RAW_ACCELEROMETERS = RAW_ACCELEROMETERS(path, D3_NORMAL_MOTORWAY)
D3_NORMAL_MOTORWAY_merged = pd.merge_asof(D3_NORMAL_MOTORWAY_RAW_GPS, D3_NORMAL_MOTORWAY_RAW_ACCELEROMETERS, on='Timestamp (seconds)', direction="nearest")
D3_NORMAL_MOTORWAY_PROC_LANE_DETECTION = PROC_LANE_DETECTION(path, D3_NORMAL_MOTORWAY)
D3_NORMAL_MOTORWAY_merged = pd.merge_asof(D3_NORMAL_MOTORWAY_merged, D3_NORMAL_MOTORWAY_PROC_LANE_DETECTION, on='Timestamp (seconds)', direction="nearest")
D3_NORMAL_MOTORWAY_PROC_VEHICLE_DETECTION = PROC_VEHICLE_DETECTION(path, D3_NORMAL_MOTORWAY)
D3_NORMAL_MOTORWAY_merged = pd.merge_asof(D3_NORMAL_MOTORWAY_merged, D3_NORMAL_MOTORWAY_PROC_VEHICLE_DETECTION, on='Timestamp (seconds)', direction="nearest")
D3_NORMAL_MOTORWAY_PROC_OPENSTREETMAP_DATA = PROC_OPENSTREETMAP_DATA(path, D3_NORMAL_MOTORWAY)
D3_NORMAL_MOTORWAY_merged = pd.merge_asof(D3_NORMAL_MOTORWAY_merged, D3_NORMAL_MOTORWAY_PROC_OPENSTREETMAP_DATA, on='Timestamp (seconds)', direction="nearest")
D3_NORMAL_MOTORWAY_SEMANTIC_ONLINE = SEMANTIC_ONLINE(path, D3_NORMAL_MOTORWAY)
D3_NORMAL_MOTORWAY_merged = pd.merge_asof(D3_NORMAL_MOTORWAY_merged, D3_NORMAL_MOTORWAY_SEMANTIC_ONLINE, on='Timestamp (seconds)', direction="nearest")


D3_AGGRESSIVE_MOTORWAY_RAW_GPS = RAW_GPS(path, D3_AGGRESSIVE_MOTORWAY)
D3_AGGRESSIVE_MOTORWAY_RAW_ACCELEROMETERS = RAW_ACCELEROMETERS(path, D3_AGGRESSIVE_MOTORWAY)
D3_AGGRESSIVE_MOTORWAY_merged = pd.merge_asof(D3_AGGRESSIVE_MOTORWAY_RAW_GPS, D3_AGGRESSIVE_MOTORWAY_RAW_ACCELEROMETERS, on='Timestamp (seconds)', direction="nearest")
D3_AGGRESSIVE_MOTORWAY_PROC_LANE_DETECTION = PROC_LANE_DETECTION(path, D3_AGGRESSIVE_MOTORWAY)
D3_AGGRESSIVE_MOTORWAY_merged = pd.merge_asof(D3_AGGRESSIVE_MOTORWAY_merged, D3_AGGRESSIVE_MOTORWAY_PROC_LANE_DETECTION, on='Timestamp (seconds)', direction="nearest")
D3_AGGRESSIVE_MOTORWAY_PROC_VEHICLE_DETECTION = PROC_VEHICLE_DETECTION(path, D3_AGGRESSIVE_MOTORWAY)
D3_AGGRESSIVE_MOTORWAY_merged = pd.merge_asof(D3_AGGRESSIVE_MOTORWAY_merged, D3_AGGRESSIVE_MOTORWAY_PROC_VEHICLE_DETECTION, on='Timestamp (seconds)', direction="nearest")
D3_AGGRESSIVE_MOTORWAY_PROC_OPENSTREETMAP_DATA = PROC_OPENSTREETMAP_DATA(path, D3_AGGRESSIVE_MOTORWAY)
D3_AGGRESSIVE_MOTORWAY_merged = pd.merge_asof(D3_AGGRESSIVE_MOTORWAY_merged, D3_AGGRESSIVE_MOTORWAY_PROC_OPENSTREETMAP_DATA, on='Timestamp (seconds)', direction="nearest")
D3_AGGRESSIVE_MOTORWAY_SEMANTIC_ONLINE = SEMANTIC_ONLINE(path, D3_AGGRESSIVE_MOTORWAY)
D3_AGGRESSIVE_MOTORWAY_merged = pd.merge_asof(D3_AGGRESSIVE_MOTORWAY_merged, D3_AGGRESSIVE_MOTORWAY_SEMANTIC_ONLINE, on='Timestamp (seconds)', direction="nearest")


D3_DROWSY_MOTORWAY_RAW_GPS = RAW_GPS(path, D3_DROWSY_MOTORWAY)
D3_DROWSY_MOTORWAY_RAW_ACCELEROMETERS = RAW_ACCELEROMETERS(path, D3_DROWSY_MOTORWAY)
D3_DROWSY_MOTORWAY_merged = pd.merge_asof(D3_DROWSY_MOTORWAY_RAW_GPS, D3_DROWSY_MOTORWAY_RAW_ACCELEROMETERS, on='Timestamp (seconds)', direction="nearest")
D3_DROWSY_MOTORWAY_PROC_LANE_DETECTION = PROC_LANE_DETECTION(path, D3_DROWSY_MOTORWAY)
D3_DROWSY_MOTORWAY_merged = pd.merge_asof(D3_DROWSY_MOTORWAY_merged, D3_DROWSY_MOTORWAY_PROC_LANE_DETECTION, on='Timestamp (seconds)', direction="nearest")
D3_DROWSY_MOTORWAY_PROC_VEHICLE_DETECTION = PROC_VEHICLE_DETECTION(path, D3_DROWSY_MOTORWAY)
D3_DROWSY_MOTORWAY_merged = pd.merge_asof(D3_DROWSY_MOTORWAY_merged, D3_DROWSY_MOTORWAY_PROC_VEHICLE_DETECTION, on='Timestamp (seconds)', direction="nearest")
D3_DROWSY_MOTORWAY_PROC_OPENSTREETMAP_DATA = PROC_OPENSTREETMAP_DATA(path, D3_DROWSY_MOTORWAY)
D3_DROWSY_MOTORWAY_merged = pd.merge_asof(D3_DROWSY_MOTORWAY_merged, D3_DROWSY_MOTORWAY_PROC_OPENSTREETMAP_DATA, on='Timestamp (seconds)', direction="nearest")
D3_DROWSY_MOTORWAY_SEMANTIC_ONLINE = SEMANTIC_ONLINE(path, D3_DROWSY_MOTORWAY)
D3_DROWSY_MOTORWAY_merged = pd.merge_asof(D3_DROWSY_MOTORWAY_merged, D3_DROWSY_MOTORWAY_SEMANTIC_ONLINE, on='Timestamp (seconds)', direction="nearest")


D3_AGGRESSIVE_SECONDARY_RAW_GPS = RAW_GPS(path, D3_AGGRESSIVE_SECONDARY)
D3_AGGRESSIVE_SECONDARY_RAW_ACCELEROMETERS = RAW_ACCELEROMETERS(path, D3_AGGRESSIVE_SECONDARY)
D3_AGGRESSIVE_SECONDARY_merged = pd.merge_asof(D3_AGGRESSIVE_SECONDARY_RAW_GPS, D3_AGGRESSIVE_SECONDARY_RAW_ACCELEROMETERS, on='Timestamp (seconds)', direction="nearest")
D3_AGGRESSIVE_SECONDARY_PROC_LANE_DETECTION = PROC_LANE_DETECTION(path, D3_AGGRESSIVE_SECONDARY)
D3_AGGRESSIVE_SECONDARY_merged = pd.merge_asof(D3_AGGRESSIVE_SECONDARY_merged, D3_AGGRESSIVE_SECONDARY_PROC_LANE_DETECTION, on='Timestamp (seconds)', direction="nearest")
D3_AGGRESSIVE_SECONDARY_PROC_VEHICLE_DETECTION = PROC_VEHICLE_DETECTION(path, D3_AGGRESSIVE_SECONDARY)
D3_AGGRESSIVE_SECONDARY_merged = pd.merge_asof(D3_AGGRESSIVE_SECONDARY_merged, D3_AGGRESSIVE_SECONDARY_PROC_VEHICLE_DETECTION, on='Timestamp (seconds)', direction="nearest")
D3_AGGRESSIVE_SECONDARY_PROC_OPENSTREETMAP_DATA = PROC_OPENSTREETMAP_DATA(path, D3_AGGRESSIVE_SECONDARY)
D3_AGGRESSIVE_SECONDARY_merged = pd.merge_asof(D3_AGGRESSIVE_SECONDARY_merged, D3_AGGRESSIVE_SECONDARY_PROC_OPENSTREETMAP_DATA, on='Timestamp (seconds)', direction="nearest")
D3_AGGRESSIVE_SECONDARY_SEMANTIC_ONLINE = SEMANTIC_ONLINE(path, D3_AGGRESSIVE_SECONDARY)
D3_AGGRESSIVE_SECONDARY_merged = pd.merge_asof(D3_AGGRESSIVE_SECONDARY_merged, D3_AGGRESSIVE_SECONDARY_SEMANTIC_ONLINE, on='Timestamp (seconds)', direction="nearest")


D3_DROWSY_SECONDARY_RAW_GPS = RAW_GPS(path, D3_DROWSY_SECONDARY)
D3_DROWSY_SECONDARY_RAW_ACCELEROMETERS = RAW_ACCELEROMETERS(path, D3_DROWSY_SECONDARY)
D3_DROWSY_SECONDARY_merged = pd.merge_asof(D3_DROWSY_SECONDARY_RAW_GPS, D3_DROWSY_SECONDARY_RAW_ACCELEROMETERS, on='Timestamp (seconds)', direction="nearest")
D3_DROWSY_SECONDARY_PROC_LANE_DETECTION = PROC_LANE_DETECTION(path, D3_DROWSY_SECONDARY)
D3_DROWSY_SECONDARY_merged = pd.merge_asof(D3_DROWSY_SECONDARY_merged, D3_DROWSY_SECONDARY_PROC_LANE_DETECTION, on='Timestamp (seconds)', direction="nearest")
D3_DROWSY_SECONDARY_PROC_VEHICLE_DETECTION = PROC_VEHICLE_DETECTION(path, D3_DROWSY_SECONDARY)
D3_DROWSY_SECONDARY_merged = pd.merge_asof(D3_DROWSY_SECONDARY_merged, D3_DROWSY_SECONDARY_PROC_VEHICLE_DETECTION, on='Timestamp (seconds)', direction="nearest")
D3_DROWSY_SECONDARY_PROC_OPENSTREETMAP_DATA = PROC_OPENSTREETMAP_DATA(path, D3_DROWSY_SECONDARY)
D3_DROWSY_SECONDARY_merged = pd.merge_asof(D3_DROWSY_SECONDARY_merged, D3_DROWSY_SECONDARY_PROC_OPENSTREETMAP_DATA, on='Timestamp (seconds)', direction="nearest")
D3_DROWSY_SECONDARY_SEMANTIC_ONLINE = SEMANTIC_ONLINE(path, D3_DROWSY_SECONDARY)
D3_DROWSY_SECONDARY_merged = pd.merge_asof(D3_DROWSY_SECONDARY_merged, D3_DROWSY_SECONDARY_SEMANTIC_ONLINE, on='Timestamp (seconds)', direction="nearest")



In [50]:
D4_NORMAL1_SECONDARY_RAW_GPS = RAW_GPS(path, D4_NORMAL1_SECONDARY)
D4_NORMAL1_SECONDARY_RAW_ACCELEROMETERS = RAW_ACCELEROMETERS(path, D4_NORMAL1_SECONDARY)
D4_NORMAL1_SECONDARY_merged = pd.merge_asof(D4_NORMAL1_SECONDARY_RAW_GPS, D4_NORMAL1_SECONDARY_RAW_ACCELEROMETERS, on='Timestamp (seconds)', direction="nearest")
D4_NORMAL1_SECONDARY_PROC_LANE_DETECTION = PROC_LANE_DETECTION(path, D4_NORMAL1_SECONDARY)
D4_NORMAL1_SECONDARY_merged = pd.merge_asof(D4_NORMAL1_SECONDARY_merged, D4_NORMAL1_SECONDARY_PROC_LANE_DETECTION, on='Timestamp (seconds)', direction="nearest")
D4_NORMAL1_SECONDARY_PROC_VEHICLE_DETECTION = PROC_VEHICLE_DETECTION(path, D4_NORMAL1_SECONDARY)
D4_NORMAL1_SECONDARY_merged = pd.merge_asof(D4_NORMAL1_SECONDARY_merged, D4_NORMAL1_SECONDARY_PROC_VEHICLE_DETECTION, on='Timestamp (seconds)', direction="nearest")
D4_NORMAL1_SECONDARY_PROC_OPENSTREETMAP_DATA = PROC_OPENSTREETMAP_DATA(path, D4_NORMAL1_SECONDARY)
D4_NORMAL1_SECONDARY_merged = pd.merge_asof(D4_NORMAL1_SECONDARY_merged, D4_NORMAL1_SECONDARY_PROC_OPENSTREETMAP_DATA, on='Timestamp (seconds)', direction="nearest")
D4_NORMAL1_SECONDARY_SEMANTIC_ONLINE = SEMANTIC_ONLINE(path, D4_NORMAL1_SECONDARY)
D4_NORMAL1_SECONDARY_merged = pd.merge_asof(D4_NORMAL1_SECONDARY_merged, D4_NORMAL1_SECONDARY_SEMANTIC_ONLINE, on='Timestamp (seconds)', direction="nearest")


D4_NORMAL2_SECONDARY_RAW_GPS = RAW_GPS(path, D4_NORMAL2_SECONDARY)
D4_NORMAL2_SECONDARY_RAW_ACCELEROMETERS = RAW_ACCELEROMETERS(path, D4_NORMAL2_SECONDARY)
D4_NORMAL2_SECONDARY_merged = pd.merge_asof(D4_NORMAL2_SECONDARY_RAW_GPS, D4_NORMAL2_SECONDARY_RAW_ACCELEROMETERS, on='Timestamp (seconds)', direction="nearest")
D4_NORMAL2_SECONDARY_PROC_LANE_DETECTION = PROC_LANE_DETECTION(path, D4_NORMAL2_SECONDARY)
D4_NORMAL2_SECONDARY_merged = pd.merge_asof(D4_NORMAL2_SECONDARY_merged, D4_NORMAL2_SECONDARY_PROC_LANE_DETECTION, on='Timestamp (seconds)', direction="nearest")
D4_NORMAL2_SECONDARY_PROC_VEHICLE_DETECTION = PROC_VEHICLE_DETECTION(path, D4_NORMAL2_SECONDARY)
D4_NORMAL2_SECONDARY_merged = pd.merge_asof(D4_NORMAL2_SECONDARY_merged, D4_NORMAL2_SECONDARY_PROC_VEHICLE_DETECTION, on='Timestamp (seconds)', direction="nearest")
D4_NORMAL2_SECONDARY_PROC_OPENSTREETMAP_DATA = PROC_OPENSTREETMAP_DATA(path, D4_NORMAL2_SECONDARY)
D4_NORMAL2_SECONDARY_merged = pd.merge_asof(D4_NORMAL2_SECONDARY_merged, D4_NORMAL2_SECONDARY_PROC_OPENSTREETMAP_DATA, on='Timestamp (seconds)', direction="nearest")
D4_NORMAL2_SECONDARY_SEMANTIC_ONLINE = SEMANTIC_ONLINE(path, D4_NORMAL2_SECONDARY)
D4_NORMAL2_SECONDARY_merged = pd.merge_asof(D4_NORMAL2_SECONDARY_merged, D4_NORMAL2_SECONDARY_SEMANTIC_ONLINE, on='Timestamp (seconds)', direction="nearest")


D4_NORMAL_MOTORWAY_RAW_GPS = RAW_GPS(path, D4_NORMAL_MOTORWAY)
D4_NORMAL_MOTORWAY_RAW_ACCELEROMETERS = RAW_ACCELEROMETERS(path, D4_NORMAL_MOTORWAY)
D4_NORMAL_MOTORWAY_merged = pd.merge_asof(D4_NORMAL_MOTORWAY_RAW_GPS, D4_NORMAL_MOTORWAY_RAW_ACCELEROMETERS, on='Timestamp (seconds)', direction="nearest")
D4_NORMAL_MOTORWAY_PROC_LANE_DETECTION = PROC_LANE_DETECTION(path, D4_NORMAL_MOTORWAY)
D4_NORMAL_MOTORWAY_merged = pd.merge_asof(D4_NORMAL_MOTORWAY_merged, D4_NORMAL_MOTORWAY_PROC_LANE_DETECTION, on='Timestamp (seconds)', direction="nearest")
D4_NORMAL_MOTORWAY_PROC_VEHICLE_DETECTION = PROC_VEHICLE_DETECTION(path, D4_NORMAL_MOTORWAY)
D4_NORMAL_MOTORWAY_merged = pd.merge_asof(D4_NORMAL_MOTORWAY_merged, D4_NORMAL_MOTORWAY_PROC_VEHICLE_DETECTION, on='Timestamp (seconds)', direction="nearest")
D4_NORMAL_MOTORWAY_PROC_OPENSTREETMAP_DATA = PROC_OPENSTREETMAP_DATA(path, D4_NORMAL_MOTORWAY)
D4_NORMAL_MOTORWAY_merged = pd.merge_asof(D4_NORMAL_MOTORWAY_merged, D4_NORMAL_MOTORWAY_PROC_OPENSTREETMAP_DATA, on='Timestamp (seconds)', direction="nearest")
D4_NORMAL_MOTORWAY_SEMANTIC_ONLINE = SEMANTIC_ONLINE(path, D4_NORMAL_MOTORWAY)
D4_NORMAL_MOTORWAY_merged = pd.merge_asof(D4_NORMAL_MOTORWAY_merged, D4_NORMAL_MOTORWAY_SEMANTIC_ONLINE, on='Timestamp (seconds)', direction="nearest")


D4_AGGRESSIVE_MOTORWAY_RAW_GPS = RAW_GPS(path, D4_AGGRESSIVE_MOTORWAY)
D4_AGGRESSIVE_MOTORWAY_RAW_ACCELEROMETERS = RAW_ACCELEROMETERS(path, D4_AGGRESSIVE_MOTORWAY)
D4_AGGRESSIVE_MOTORWAY_merged = pd.merge_asof(D4_AGGRESSIVE_MOTORWAY_RAW_GPS, D4_AGGRESSIVE_MOTORWAY_RAW_ACCELEROMETERS, on='Timestamp (seconds)', direction="nearest")
D4_AGGRESSIVE_MOTORWAY_PROC_LANE_DETECTION = PROC_LANE_DETECTION(path, D4_AGGRESSIVE_MOTORWAY)
D4_AGGRESSIVE_MOTORWAY_merged = pd.merge_asof(D4_AGGRESSIVE_MOTORWAY_merged, D4_AGGRESSIVE_MOTORWAY_PROC_LANE_DETECTION, on='Timestamp (seconds)', direction="nearest")
D4_AGGRESSIVE_MOTORWAY_PROC_VEHICLE_DETECTION = PROC_VEHICLE_DETECTION(path, D4_AGGRESSIVE_MOTORWAY)
D4_AGGRESSIVE_MOTORWAY_merged = pd.merge_asof(D4_AGGRESSIVE_MOTORWAY_merged, D4_AGGRESSIVE_MOTORWAY_PROC_VEHICLE_DETECTION, on='Timestamp (seconds)', direction="nearest")
D4_AGGRESSIVE_MOTORWAY_PROC_OPENSTREETMAP_DATA = PROC_OPENSTREETMAP_DATA(path, D4_AGGRESSIVE_MOTORWAY)
D4_AGGRESSIVE_MOTORWAY_merged = pd.merge_asof(D4_AGGRESSIVE_MOTORWAY_merged, D4_AGGRESSIVE_MOTORWAY_PROC_OPENSTREETMAP_DATA, on='Timestamp (seconds)', direction="nearest")
D4_AGGRESSIVE_MOTORWAY_SEMANTIC_ONLINE = SEMANTIC_ONLINE(path, D4_AGGRESSIVE_MOTORWAY)
D4_AGGRESSIVE_MOTORWAY_merged = pd.merge_asof(D4_AGGRESSIVE_MOTORWAY_merged, D4_AGGRESSIVE_MOTORWAY_SEMANTIC_ONLINE, on='Timestamp (seconds)', direction="nearest")


D4_DROWSY_MOTORWAY_RAW_GPS = RAW_GPS(path, D4_DROWSY_MOTORWAY)
D4_DROWSY_MOTORWAY_RAW_ACCELEROMETERS = RAW_ACCELEROMETERS(path, D4_DROWSY_MOTORWAY)
D4_DROWSY_MOTORWAY_merged = pd.merge_asof(D4_DROWSY_MOTORWAY_RAW_GPS, D4_DROWSY_MOTORWAY_RAW_ACCELEROMETERS, on='Timestamp (seconds)', direction="nearest")
D4_DROWSY_MOTORWAY_PROC_LANE_DETECTION = PROC_LANE_DETECTION(path, D4_DROWSY_MOTORWAY)
D4_DROWSY_MOTORWAY_merged = pd.merge_asof(D4_DROWSY_MOTORWAY_merged, D4_DROWSY_MOTORWAY_PROC_LANE_DETECTION, on='Timestamp (seconds)', direction="nearest")
D4_DROWSY_MOTORWAY_PROC_VEHICLE_DETECTION = PROC_VEHICLE_DETECTION(path, D4_DROWSY_MOTORWAY)
D4_DROWSY_MOTORWAY_merged = pd.merge_asof(D4_DROWSY_MOTORWAY_merged, D4_DROWSY_MOTORWAY_PROC_VEHICLE_DETECTION, on='Timestamp (seconds)', direction="nearest")
D4_DROWSY_MOTORWAY_PROC_OPENSTREETMAP_DATA = PROC_OPENSTREETMAP_DATA(path, D4_DROWSY_MOTORWAY)
D4_DROWSY_MOTORWAY_merged = pd.merge_asof(D4_DROWSY_MOTORWAY_merged, D4_DROWSY_MOTORWAY_PROC_OPENSTREETMAP_DATA, on='Timestamp (seconds)', direction="nearest")
D4_DROWSY_MOTORWAY_SEMANTIC_ONLINE = SEMANTIC_ONLINE(path, D4_DROWSY_MOTORWAY)
D4_DROWSY_MOTORWAY_merged = pd.merge_asof(D4_DROWSY_MOTORWAY_merged, D4_DROWSY_MOTORWAY_SEMANTIC_ONLINE, on='Timestamp (seconds)', direction="nearest")


D4_AGGRESSIVE_SECONDARY_RAW_GPS = RAW_GPS(path, D4_AGGRESSIVE_SECONDARY)
D4_AGGRESSIVE_SECONDARY_RAW_ACCELEROMETERS = RAW_ACCELEROMETERS(path, D4_AGGRESSIVE_SECONDARY)
D4_AGGRESSIVE_SECONDARY_merged = pd.merge_asof(D4_AGGRESSIVE_SECONDARY_RAW_GPS, D4_AGGRESSIVE_SECONDARY_RAW_ACCELEROMETERS, on='Timestamp (seconds)', direction="nearest")
D4_AGGRESSIVE_SECONDARY_PROC_LANE_DETECTION = PROC_LANE_DETECTION(path, D4_AGGRESSIVE_SECONDARY)
D4_AGGRESSIVE_SECONDARY_merged = pd.merge_asof(D4_AGGRESSIVE_SECONDARY_merged, D4_AGGRESSIVE_SECONDARY_PROC_LANE_DETECTION, on='Timestamp (seconds)', direction="nearest")
D4_AGGRESSIVE_SECONDARY_PROC_VEHICLE_DETECTION = PROC_VEHICLE_DETECTION(path, D4_AGGRESSIVE_SECONDARY)
D4_AGGRESSIVE_SECONDARY_merged = pd.merge_asof(D4_AGGRESSIVE_SECONDARY_merged, D4_AGGRESSIVE_SECONDARY_PROC_VEHICLE_DETECTION, on='Timestamp (seconds)', direction="nearest")
D4_AGGRESSIVE_SECONDARY_PROC_OPENSTREETMAP_DATA = PROC_OPENSTREETMAP_DATA(path, D4_AGGRESSIVE_SECONDARY)
D4_AGGRESSIVE_SECONDARY_merged = pd.merge_asof(D4_AGGRESSIVE_SECONDARY_merged, D4_AGGRESSIVE_SECONDARY_PROC_OPENSTREETMAP_DATA, on='Timestamp (seconds)', direction="nearest")
D4_AGGRESSIVE_SECONDARY_SEMANTIC_ONLINE = SEMANTIC_ONLINE(path, D4_AGGRESSIVE_SECONDARY)
D4_AGGRESSIVE_SECONDARY_merged = pd.merge_asof(D4_AGGRESSIVE_SECONDARY_merged, D4_AGGRESSIVE_SECONDARY_SEMANTIC_ONLINE, on='Timestamp (seconds)', direction="nearest")


D4_DROWSY_SECONDARY_RAW_GPS = RAW_GPS(path, D4_DROWSY_SECONDARY)
D4_DROWSY_SECONDARY_RAW_ACCELEROMETERS = RAW_ACCELEROMETERS(path, D4_DROWSY_SECONDARY)
D4_DROWSY_SECONDARY_merged = pd.merge_asof(D4_DROWSY_SECONDARY_RAW_GPS, D4_DROWSY_SECONDARY_RAW_ACCELEROMETERS, on='Timestamp (seconds)', direction="nearest")
D4_DROWSY_SECONDARY_PROC_LANE_DETECTION = PROC_LANE_DETECTION(path, D4_DROWSY_SECONDARY)
D4_DROWSY_SECONDARY_merged = pd.merge_asof(D4_DROWSY_SECONDARY_merged, D4_DROWSY_SECONDARY_PROC_LANE_DETECTION, on='Timestamp (seconds)', direction="nearest")
D4_DROWSY_SECONDARY_PROC_VEHICLE_DETECTION = PROC_VEHICLE_DETECTION(path, D4_DROWSY_SECONDARY)
D4_DROWSY_SECONDARY_merged = pd.merge_asof(D4_DROWSY_SECONDARY_merged, D4_DROWSY_SECONDARY_PROC_VEHICLE_DETECTION, on='Timestamp (seconds)', direction="nearest")
D4_DROWSY_SECONDARY_PROC_OPENSTREETMAP_DATA = PROC_OPENSTREETMAP_DATA(path, D4_DROWSY_SECONDARY)
D4_DROWSY_SECONDARY_merged = pd.merge_asof(D4_DROWSY_SECONDARY_merged, D4_DROWSY_SECONDARY_PROC_OPENSTREETMAP_DATA, on='Timestamp (seconds)', direction="nearest")
D4_DROWSY_SECONDARY_SEMANTIC_ONLINE = SEMANTIC_ONLINE(path, D4_DROWSY_SECONDARY)
D4_DROWSY_SECONDARY_merged = pd.merge_asof(D4_DROWSY_SECONDARY_merged, D4_DROWSY_SECONDARY_SEMANTIC_ONLINE, on='Timestamp (seconds)', direction="nearest")

In [51]:
D5_NORMAL1_SECONDARY_RAW_GPS = RAW_GPS(path, D5_NORMAL1_SECONDARY)
D5_NORMAL1_SECONDARY_RAW_ACCELEROMETERS = RAW_ACCELEROMETERS(path, D5_NORMAL1_SECONDARY)
D5_NORMAL1_SECONDARY_merged = pd.merge_asof(D5_NORMAL1_SECONDARY_RAW_GPS, D5_NORMAL1_SECONDARY_RAW_ACCELEROMETERS, on='Timestamp (seconds)', direction="nearest")
D5_NORMAL1_SECONDARY_PROC_LANE_DETECTION = PROC_LANE_DETECTION(path, D5_NORMAL1_SECONDARY)
D5_NORMAL1_SECONDARY_merged = pd.merge_asof(D5_NORMAL1_SECONDARY_merged, D5_NORMAL1_SECONDARY_PROC_LANE_DETECTION, on='Timestamp (seconds)', direction="nearest")
D5_NORMAL1_SECONDARY_PROC_VEHICLE_DETECTION = PROC_VEHICLE_DETECTION(path, D5_NORMAL1_SECONDARY)
D5_NORMAL1_SECONDARY_merged = pd.merge_asof(D5_NORMAL1_SECONDARY_merged, D5_NORMAL1_SECONDARY_PROC_VEHICLE_DETECTION, on='Timestamp (seconds)', direction="nearest")
D5_NORMAL1_SECONDARY_PROC_OPENSTREETMAP_DATA = PROC_OPENSTREETMAP_DATA(path, D5_NORMAL1_SECONDARY)
D5_NORMAL1_SECONDARY_merged = pd.merge_asof(D5_NORMAL1_SECONDARY_merged, D5_NORMAL1_SECONDARY_PROC_OPENSTREETMAP_DATA, on='Timestamp (seconds)', direction="nearest")
D5_NORMAL1_SECONDARY_SEMANTIC_ONLINE = SEMANTIC_ONLINE(path, D5_NORMAL1_SECONDARY)
D5_NORMAL1_SECONDARY_merged = pd.merge_asof(D5_NORMAL1_SECONDARY_merged, D5_NORMAL1_SECONDARY_SEMANTIC_ONLINE, on='Timestamp (seconds)', direction="nearest")


D5_NORMAL2_SECONDARY_RAW_GPS = RAW_GPS(path, D5_NORMAL2_SECONDARY)
D5_NORMAL2_SECONDARY_RAW_ACCELEROMETERS = RAW_ACCELEROMETERS(path, D5_NORMAL2_SECONDARY)
D5_NORMAL2_SECONDARY_merged = pd.merge_asof(D5_NORMAL2_SECONDARY_RAW_GPS, D5_NORMAL2_SECONDARY_RAW_ACCELEROMETERS, on='Timestamp (seconds)', direction="nearest")
D5_NORMAL2_SECONDARY_PROC_LANE_DETECTION = PROC_LANE_DETECTION(path, D5_NORMAL2_SECONDARY)
D5_NORMAL2_SECONDARY_merged = pd.merge_asof(D5_NORMAL2_SECONDARY_merged, D5_NORMAL2_SECONDARY_PROC_LANE_DETECTION, on='Timestamp (seconds)', direction="nearest")
D5_NORMAL2_SECONDARY_PROC_VEHICLE_DETECTION = PROC_VEHICLE_DETECTION(path, D5_NORMAL2_SECONDARY)
D5_NORMAL2_SECONDARY_merged = pd.merge_asof(D5_NORMAL2_SECONDARY_merged, D5_NORMAL2_SECONDARY_PROC_VEHICLE_DETECTION, on='Timestamp (seconds)', direction="nearest")
D5_NORMAL2_SECONDARY_PROC_OPENSTREETMAP_DATA = PROC_OPENSTREETMAP_DATA(path, D5_NORMAL2_SECONDARY)
D5_NORMAL2_SECONDARY_merged = pd.merge_asof(D5_NORMAL2_SECONDARY_merged, D5_NORMAL2_SECONDARY_PROC_OPENSTREETMAP_DATA, on='Timestamp (seconds)', direction="nearest")
D5_NORMAL2_SECONDARY_SEMANTIC_ONLINE = SEMANTIC_ONLINE(path, D5_NORMAL2_SECONDARY)
D5_NORMAL2_SECONDARY_merged = pd.merge_asof(D5_NORMAL2_SECONDARY_merged, D5_NORMAL2_SECONDARY_SEMANTIC_ONLINE, on='Timestamp (seconds)', direction="nearest")


D5_NORMAL_MOTORWAY_RAW_GPS = RAW_GPS(path, D5_NORMAL_MOTORWAY)
D5_NORMAL_MOTORWAY_RAW_ACCELEROMETERS = RAW_ACCELEROMETERS(path, D5_NORMAL_MOTORWAY)
D5_NORMAL_MOTORWAY_merged = pd.merge_asof(D5_NORMAL_MOTORWAY_RAW_GPS, D5_NORMAL_MOTORWAY_RAW_ACCELEROMETERS, on='Timestamp (seconds)', direction="nearest")
D5_NORMAL_MOTORWAY_PROC_LANE_DETECTION = PROC_LANE_DETECTION(path, D5_NORMAL_MOTORWAY)
D5_NORMAL_MOTORWAY_merged = pd.merge_asof(D5_NORMAL_MOTORWAY_merged, D5_NORMAL_MOTORWAY_PROC_LANE_DETECTION, on='Timestamp (seconds)', direction="nearest")
D5_NORMAL_MOTORWAY_PROC_VEHICLE_DETECTION = PROC_VEHICLE_DETECTION(path, D5_NORMAL_MOTORWAY)
D5_NORMAL_MOTORWAY_merged = pd.merge_asof(D5_NORMAL_MOTORWAY_merged, D5_NORMAL_MOTORWAY_PROC_VEHICLE_DETECTION, on='Timestamp (seconds)', direction="nearest")
D5_NORMAL_MOTORWAY_PROC_OPENSTREETMAP_DATA = PROC_OPENSTREETMAP_DATA(path, D5_NORMAL_MOTORWAY)
D5_NORMAL_MOTORWAY_merged = pd.merge_asof(D5_NORMAL_MOTORWAY_merged, D5_NORMAL_MOTORWAY_PROC_OPENSTREETMAP_DATA, on='Timestamp (seconds)', direction="nearest")
D5_NORMAL_MOTORWAY_SEMANTIC_ONLINE = SEMANTIC_ONLINE(path, D5_NORMAL_MOTORWAY)
D5_NORMAL_MOTORWAY_merged = pd.merge_asof(D5_NORMAL_MOTORWAY_merged, D5_NORMAL_MOTORWAY_SEMANTIC_ONLINE, on='Timestamp (seconds)', direction="nearest")


D5_AGGRESSIVE_MOTORWAY_RAW_GPS = RAW_GPS(path, D5_AGGRESSIVE_MOTORWAY)
D5_AGGRESSIVE_MOTORWAY_RAW_ACCELEROMETERS = RAW_ACCELEROMETERS(path, D5_AGGRESSIVE_MOTORWAY)
D5_AGGRESSIVE_MOTORWAY_merged = pd.merge_asof(D5_AGGRESSIVE_MOTORWAY_RAW_GPS, D5_AGGRESSIVE_MOTORWAY_RAW_ACCELEROMETERS, on='Timestamp (seconds)', direction="nearest")
D5_AGGRESSIVE_MOTORWAY_PROC_LANE_DETECTION = PROC_LANE_DETECTION(path, D5_AGGRESSIVE_MOTORWAY)
D5_AGGRESSIVE_MOTORWAY_merged = pd.merge_asof(D5_AGGRESSIVE_MOTORWAY_merged, D5_AGGRESSIVE_MOTORWAY_PROC_LANE_DETECTION, on='Timestamp (seconds)', direction="nearest")
D5_AGGRESSIVE_MOTORWAY_PROC_VEHICLE_DETECTION = PROC_VEHICLE_DETECTION(path, D5_AGGRESSIVE_MOTORWAY)
D5_AGGRESSIVE_MOTORWAY_merged = pd.merge_asof(D5_AGGRESSIVE_MOTORWAY_merged, D5_AGGRESSIVE_MOTORWAY_PROC_VEHICLE_DETECTION, on='Timestamp (seconds)', direction="nearest")
D5_AGGRESSIVE_MOTORWAY_PROC_OPENSTREETMAP_DATA = PROC_OPENSTREETMAP_DATA(path, D5_AGGRESSIVE_MOTORWAY)
D5_AGGRESSIVE_MOTORWAY_merged = pd.merge_asof(D5_AGGRESSIVE_MOTORWAY_merged, D5_AGGRESSIVE_MOTORWAY_PROC_OPENSTREETMAP_DATA, on='Timestamp (seconds)', direction="nearest")
D5_AGGRESSIVE_MOTORWAY_SEMANTIC_ONLINE = SEMANTIC_ONLINE(path, D5_AGGRESSIVE_MOTORWAY)
D5_AGGRESSIVE_MOTORWAY_merged = pd.merge_asof(D5_AGGRESSIVE_MOTORWAY_merged, D5_AGGRESSIVE_MOTORWAY_SEMANTIC_ONLINE, on='Timestamp (seconds)', direction="nearest")


D5_DROWSY_MOTORWAY_RAW_GPS = RAW_GPS(path, D5_DROWSY_MOTORWAY)
D5_DROWSY_MOTORWAY_RAW_ACCELEROMETERS = RAW_ACCELEROMETERS(path, D5_DROWSY_MOTORWAY)
D5_DROWSY_MOTORWAY_merged = pd.merge_asof(D5_DROWSY_MOTORWAY_RAW_GPS, D5_DROWSY_MOTORWAY_RAW_ACCELEROMETERS, on='Timestamp (seconds)', direction="nearest")
D5_DROWSY_MOTORWAY_PROC_LANE_DETECTION = PROC_LANE_DETECTION(path, D5_DROWSY_MOTORWAY)
D5_DROWSY_MOTORWAY_merged = pd.merge_asof(D5_DROWSY_MOTORWAY_merged, D5_DROWSY_MOTORWAY_PROC_LANE_DETECTION, on='Timestamp (seconds)', direction="nearest")
D5_DROWSY_MOTORWAY_PROC_VEHICLE_DETECTION = PROC_VEHICLE_DETECTION(path, D5_DROWSY_MOTORWAY)
D5_DROWSY_MOTORWAY_merged = pd.merge_asof(D5_DROWSY_MOTORWAY_merged, D5_DROWSY_MOTORWAY_PROC_VEHICLE_DETECTION, on='Timestamp (seconds)', direction="nearest")
D5_DROWSY_MOTORWAY_PROC_OPENSTREETMAP_DATA = PROC_OPENSTREETMAP_DATA(path, D5_DROWSY_MOTORWAY)
D5_DROWSY_MOTORWAY_merged = pd.merge_asof(D5_DROWSY_MOTORWAY_merged, D5_DROWSY_MOTORWAY_PROC_OPENSTREETMAP_DATA, on='Timestamp (seconds)', direction="nearest")
D5_DROWSY_MOTORWAY_SEMANTIC_ONLINE = SEMANTIC_ONLINE(path, D5_DROWSY_MOTORWAY)
D5_DROWSY_MOTORWAY_merged = pd.merge_asof(D5_DROWSY_MOTORWAY_merged, D5_DROWSY_MOTORWAY_SEMANTIC_ONLINE, on='Timestamp (seconds)', direction="nearest")


D5_AGGRESSIVE_SECONDARY_RAW_GPS = RAW_GPS(path, D5_AGGRESSIVE_SECONDARY)
D5_AGGRESSIVE_SECONDARY_RAW_ACCELEROMETERS = RAW_ACCELEROMETERS(path, D5_AGGRESSIVE_SECONDARY)
D5_AGGRESSIVE_SECONDARY_merged = pd.merge_asof(D5_AGGRESSIVE_SECONDARY_RAW_GPS, D5_AGGRESSIVE_SECONDARY_RAW_ACCELEROMETERS, on='Timestamp (seconds)', direction="nearest")
D5_AGGRESSIVE_SECONDARY_PROC_LANE_DETECTION = PROC_LANE_DETECTION(path, D5_AGGRESSIVE_SECONDARY)
D5_AGGRESSIVE_SECONDARY_merged = pd.merge_asof(D5_AGGRESSIVE_SECONDARY_merged, D5_AGGRESSIVE_SECONDARY_PROC_LANE_DETECTION, on='Timestamp (seconds)', direction="nearest")
D5_AGGRESSIVE_SECONDARY_PROC_VEHICLE_DETECTION = PROC_VEHICLE_DETECTION(path, D5_AGGRESSIVE_SECONDARY)
D5_AGGRESSIVE_SECONDARY_merged = pd.merge_asof(D5_AGGRESSIVE_SECONDARY_merged, D5_AGGRESSIVE_SECONDARY_PROC_VEHICLE_DETECTION, on='Timestamp (seconds)', direction="nearest")
D5_AGGRESSIVE_SECONDARY_PROC_OPENSTREETMAP_DATA = PROC_OPENSTREETMAP_DATA(path, D5_AGGRESSIVE_SECONDARY)
D5_AGGRESSIVE_SECONDARY_merged = pd.merge_asof(D5_AGGRESSIVE_SECONDARY_merged, D5_AGGRESSIVE_SECONDARY_PROC_OPENSTREETMAP_DATA, on='Timestamp (seconds)', direction="nearest")
D5_AGGRESSIVE_SECONDARY_SEMANTIC_ONLINE = SEMANTIC_ONLINE(path, D5_AGGRESSIVE_SECONDARY)
D5_AGGRESSIVE_SECONDARY_merged = pd.merge_asof(D5_AGGRESSIVE_SECONDARY_merged, D5_AGGRESSIVE_SECONDARY_SEMANTIC_ONLINE, on='Timestamp (seconds)', direction="nearest")


D5_DROWSY_SECONDARY_RAW_GPS = RAW_GPS(path, D5_DROWSY_SECONDARY)
D5_DROWSY_SECONDARY_RAW_ACCELEROMETERS = RAW_ACCELEROMETERS(path, D5_DROWSY_SECONDARY)
D5_DROWSY_SECONDARY_merged = pd.merge_asof(D5_DROWSY_SECONDARY_RAW_GPS, D5_DROWSY_SECONDARY_RAW_ACCELEROMETERS, on='Timestamp (seconds)', direction="nearest")
D5_DROWSY_SECONDARY_PROC_LANE_DETECTION = PROC_LANE_DETECTION(path, D5_DROWSY_SECONDARY)
D5_DROWSY_SECONDARY_merged = pd.merge_asof(D5_DROWSY_SECONDARY_merged, D5_DROWSY_SECONDARY_PROC_LANE_DETECTION, on='Timestamp (seconds)', direction="nearest")
D5_DROWSY_SECONDARY_PROC_VEHICLE_DETECTION = PROC_VEHICLE_DETECTION(path, D5_DROWSY_SECONDARY)
D5_DROWSY_SECONDARY_merged = pd.merge_asof(D5_DROWSY_SECONDARY_merged, D5_DROWSY_SECONDARY_PROC_VEHICLE_DETECTION, on='Timestamp (seconds)', direction="nearest")
D5_DROWSY_SECONDARY_PROC_OPENSTREETMAP_DATA = PROC_OPENSTREETMAP_DATA(path, D5_DROWSY_SECONDARY)
D5_DROWSY_SECONDARY_merged = pd.merge_asof(D5_DROWSY_SECONDARY_merged, D5_DROWSY_SECONDARY_PROC_OPENSTREETMAP_DATA, on='Timestamp (seconds)', direction="nearest")
D5_DROWSY_SECONDARY_SEMANTIC_ONLINE = SEMANTIC_ONLINE(path, D5_DROWSY_SECONDARY)
D5_DROWSY_SECONDARY_merged = pd.merge_asof(D5_DROWSY_SECONDARY_merged, D5_DROWSY_SECONDARY_SEMANTIC_ONLINE, on='Timestamp (seconds)', direction="nearest")

In [52]:
D6_NORMAL_SECONDARY_RAW_GPS = RAW_GPS(path, D6_NORMAL_SECONDARY)
D6_NORMAL_SECONDARY_RAW_ACCELEROMETERS = RAW_ACCELEROMETERS(path, D6_NORMAL_SECONDARY)
D6_NORMAL_SECONDARY_merged = pd.merge_asof(D6_NORMAL_SECONDARY_RAW_GPS, D6_NORMAL_SECONDARY_RAW_ACCELEROMETERS, on='Timestamp (seconds)', direction="nearest")
D6_NORMAL_SECONDARY_PROC_LANE_DETECTION = PROC_LANE_DETECTION(path, D6_NORMAL_SECONDARY)
D6_NORMAL_SECONDARY_merged = pd.merge_asof(D6_NORMAL_SECONDARY_merged, D6_NORMAL_SECONDARY_PROC_LANE_DETECTION, on='Timestamp (seconds)', direction="nearest")
D6_NORMAL_SECONDARY_PROC_VEHICLE_DETECTION = PROC_VEHICLE_DETECTION(path, D6_NORMAL_SECONDARY)
D6_NORMAL_SECONDARY_merged = pd.merge_asof(D6_NORMAL_SECONDARY_merged, D6_NORMAL_SECONDARY_PROC_VEHICLE_DETECTION, on='Timestamp (seconds)', direction="nearest")
D6_NORMAL_SECONDARY_PROC_OPENSTREETMAP_DATA = PROC_OPENSTREETMAP_DATA(path, D6_NORMAL_SECONDARY)
D6_NORMAL_SECONDARY_merged = pd.merge_asof(D6_NORMAL_SECONDARY_merged, D6_NORMAL_SECONDARY_PROC_OPENSTREETMAP_DATA, on='Timestamp (seconds)', direction="nearest")
D6_NORMAL_SECONDARY_SEMANTIC_ONLINE = SEMANTIC_ONLINE(path, D6_NORMAL_SECONDARY)
D6_NORMAL_SECONDARY_merged = pd.merge_asof(D6_NORMAL_SECONDARY_merged, D6_NORMAL_SECONDARY_SEMANTIC_ONLINE, on='Timestamp (seconds)', direction="nearest")

D6_AGGRESSIVE_MOTORWAY_RAW_GPS = RAW_GPS(path, D6_AGGRESSIVE_MOTORWAY)
D6_AGGRESSIVE_MOTORWAY_RAW_ACCELEROMETERS = RAW_ACCELEROMETERS(path, D6_AGGRESSIVE_MOTORWAY)
D6_AGGRESSIVE_MOTORWAY_merged = pd.merge_asof(D6_AGGRESSIVE_MOTORWAY_RAW_GPS, D6_AGGRESSIVE_MOTORWAY_RAW_ACCELEROMETERS, on='Timestamp (seconds)', direction="nearest")
D6_AGGRESSIVE_MOTORWAY_PROC_LANE_DETECTION = PROC_LANE_DETECTION(path, D6_AGGRESSIVE_MOTORWAY)
D6_AGGRESSIVE_MOTORWAY_merged = pd.merge_asof(D6_AGGRESSIVE_MOTORWAY_merged, D6_AGGRESSIVE_MOTORWAY_PROC_LANE_DETECTION, on='Timestamp (seconds)', direction="nearest")
D6_AGGRESSIVE_MOTORWAY_PROC_VEHICLE_DETECTION = PROC_VEHICLE_DETECTION(path, D6_AGGRESSIVE_MOTORWAY)
D6_AGGRESSIVE_MOTORWAY_merged = pd.merge_asof(D6_AGGRESSIVE_MOTORWAY_merged, D6_AGGRESSIVE_MOTORWAY_PROC_VEHICLE_DETECTION, on='Timestamp (seconds)', direction="nearest")
D6_AGGRESSIVE_MOTORWAY_PROC_OPENSTREETMAP_DATA = PROC_OPENSTREETMAP_DATA(path, D6_AGGRESSIVE_MOTORWAY)
D6_AGGRESSIVE_MOTORWAY_merged = pd.merge_asof(D6_AGGRESSIVE_MOTORWAY_merged, D6_AGGRESSIVE_MOTORWAY_PROC_OPENSTREETMAP_DATA, on='Timestamp (seconds)', direction="nearest")
D6_AGGRESSIVE_MOTORWAY_SEMANTIC_ONLINE = SEMANTIC_ONLINE(path, D6_AGGRESSIVE_MOTORWAY)
D6_AGGRESSIVE_MOTORWAY_merged = pd.merge_asof(D6_AGGRESSIVE_MOTORWAY_merged, D6_AGGRESSIVE_MOTORWAY_SEMANTIC_ONLINE, on='Timestamp (seconds)', direction="nearest")

D6_DROWSY_MOTORWAY_RAW_GPS = RAW_GPS(path, D6_DROWSY_MOTORWAY).sort_values(by='Timestamp (seconds)')
D6_DROWSY_MOTORWAY_RAW_ACCELEROMETERS = RAW_ACCELEROMETERS(path, D6_DROWSY_MOTORWAY).sort_values(by='Timestamp (seconds)')
D6_DROWSY_MOTORWAY_merged = pd.merge_asof(D6_DROWSY_MOTORWAY_RAW_GPS, D6_DROWSY_MOTORWAY_RAW_ACCELEROMETERS, on='Timestamp (seconds)', direction="nearest")
D6_DROWSY_MOTORWAY_PROC_LANE_DETECTION = PROC_LANE_DETECTION(path, D6_DROWSY_MOTORWAY)
D6_DROWSY_MOTORWAY_merged = pd.merge_asof(D6_DROWSY_MOTORWAY_merged, D6_DROWSY_MOTORWAY_PROC_LANE_DETECTION, on='Timestamp (seconds)', direction="nearest")
D6_DROWSY_MOTORWAY_PROC_VEHICLE_DETECTION = PROC_VEHICLE_DETECTION(path, D6_DROWSY_MOTORWAY).sort_values(by='Timestamp (seconds)')
D6_DROWSY_MOTORWAY_merged = pd.merge_asof(D6_DROWSY_MOTORWAY_merged, D6_DROWSY_MOTORWAY_PROC_VEHICLE_DETECTION, on='Timestamp (seconds)', direction="nearest")
D6_DROWSY_MOTORWAY_PROC_OPENSTREETMAP_DATA = PROC_OPENSTREETMAP_DATA(path, D6_DROWSY_MOTORWAY)
D6_DROWSY_MOTORWAY_merged = pd.merge_asof(D6_DROWSY_MOTORWAY_merged, D6_DROWSY_MOTORWAY_PROC_OPENSTREETMAP_DATA, on='Timestamp (seconds)', direction="nearest")
D6_DROWSY_MOTORWAY_SEMANTIC_ONLINE = SEMANTIC_ONLINE(path, D6_DROWSY_MOTORWAY)
D6_DROWSY_MOTORWAY_merged = pd.merge_asof(D6_DROWSY_MOTORWAY_merged, D6_DROWSY_MOTORWAY_SEMANTIC_ONLINE, on='Timestamp (seconds)', direction="nearest")

D6_DROWSY_SECONDARY_RAW_GPS = RAW_GPS(path, D6_DROWSY_SECONDARY)
D6_DROWSY_SECONDARY_RAW_ACCELEROMETERS = RAW_ACCELEROMETERS(path, D6_DROWSY_SECONDARY)
D6_DROWSY_SECONDARY_merged = pd.merge_asof(D6_DROWSY_SECONDARY_RAW_GPS, D6_DROWSY_SECONDARY_RAW_ACCELEROMETERS, on='Timestamp (seconds)', direction="nearest")
D6_DROWSY_SECONDARY_PROC_LANE_DETECTION = PROC_LANE_DETECTION(path, D6_DROWSY_SECONDARY)
D6_DROWSY_SECONDARY_merged = pd.merge_asof(D6_DROWSY_SECONDARY_merged, D6_DROWSY_SECONDARY_PROC_LANE_DETECTION, on='Timestamp (seconds)', direction="nearest")
D6_DROWSY_SECONDARY_PROC_VEHICLE_DETECTION = PROC_VEHICLE_DETECTION(path, D6_DROWSY_SECONDARY)
D6_DROWSY_SECONDARY_merged = pd.merge_asof(D6_DROWSY_SECONDARY_merged, D6_DROWSY_SECONDARY_PROC_VEHICLE_DETECTION, on='Timestamp (seconds)', direction="nearest")
D6_DROWSY_SECONDARY_PROC_OPENSTREETMAP_DATA = PROC_OPENSTREETMAP_DATA(path, D6_DROWSY_SECONDARY)
D6_DROWSY_SECONDARY_merged = pd.merge_asof(D6_DROWSY_SECONDARY_merged, D6_DROWSY_SECONDARY_PROC_OPENSTREETMAP_DATA, on='Timestamp (seconds)', direction="nearest")
D6_DROWSY_SECONDARY_SEMANTIC_ONLINE = SEMANTIC_ONLINE(path, D6_DROWSY_SECONDARY)
D6_DROWSY_SECONDARY_merged = pd.merge_asof(D6_DROWSY_SECONDARY_merged, D6_DROWSY_SECONDARY_SEMANTIC_ONLINE, on='Timestamp (seconds)', direction="nearest")



In [54]:
def RAW_ACCELEROMETERS(path, driver_condition):
    
    driver_condition = path+driver_condition

    RAW_ACCELEROMETERS = pd.read_csv (driver_condition+'/RAW_ACCELEROMETERS.txt', sep=" ")
    RAW_ACCELEROMETERS.columns = [
        'Timestamp (seconds)',
        'Boolean of system activated (1 if >50km/h)',
        'Acceleration in X (Gs)',
        'Acceleration in Y (Gs)',
        'Acceleration in Z (Gs)',
        'Acceleration in X filtered by KF (Gs)',
        'Acceleration in Y filtered by KF (Gs)',
        'Acceleration in Z filtered by KF (Gs)',
        'Roll (degrees)',
        'Pitch (degrees)',
        'Yaw (degrees)']
    return RAW_ACCELEROMETERS

In [55]:
D6_NORMAL_MOTORWAY_RAW_GPS = RAW_GPS(path, D6_NORMAL_MOTORWAY)
D6_NORMAL_MOTORWAY_RAW_ACCELEROMETERS = RAW_ACCELEROMETERS(path, D6_NORMAL_MOTORWAY)
D6_NORMAL_MOTORWAY_merged = pd.merge_asof(D6_NORMAL_MOTORWAY_RAW_GPS, D6_NORMAL_MOTORWAY_RAW_ACCELEROMETERS, on='Timestamp (seconds)', direction="nearest")
D6_NORMAL_MOTORWAY_PROC_LANE_DETECTION = PROC_LANE_DETECTION(path, D6_NORMAL_MOTORWAY)
D6_NORMAL_MOTORWAY_merged = pd.merge_asof(D6_NORMAL_MOTORWAY_merged, D6_NORMAL_MOTORWAY_PROC_LANE_DETECTION, on='Timestamp (seconds)', direction="nearest")
D6_NORMAL_MOTORWAY_PROC_VEHICLE_DETECTION = PROC_VEHICLE_DETECTION(path, D6_NORMAL_MOTORWAY)
D6_NORMAL_MOTORWAY_merged = pd.merge_asof(D6_NORMAL_MOTORWAY_merged, D6_NORMAL_MOTORWAY_PROC_VEHICLE_DETECTION, on='Timestamp (seconds)', direction="nearest")
D6_NORMAL_MOTORWAY_PROC_OPENSTREETMAP_DATA = PROC_OPENSTREETMAP_DATA(path, D6_NORMAL_MOTORWAY)
D6_NORMAL_MOTORWAY_merged = pd.merge_asof(D6_NORMAL_MOTORWAY_merged, D6_NORMAL_MOTORWAY_PROC_OPENSTREETMAP_DATA, on='Timestamp (seconds)', direction="nearest")
D6_NORMAL_MOTORWAY_SEMANTIC_ONLINE = SEMANTIC_ONLINE(path, D6_NORMAL_MOTORWAY)
D6_NORMAL_MOTORWAY_merged = pd.merge_asof(D6_NORMAL_MOTORWAY_merged, D6_NORMAL_MOTORWAY_SEMANTIC_ONLINE, on='Timestamp (seconds)', direction="nearest")

In [87]:
D1_merged = pd.concat([D1_NORMAL1_SECONDARY_merged,
D1_NORMAL2_SECONDARY_merged,
D1_NORMAL_MOTORWAY_merged,
D1_AGGRESSIVE_MOTORWAY_merged,
D1_DROWSY_MOTORWAY_merged,
D1_AGGRESSIVE_SECONDARY_merged,
D1_DROWSY_SECONDARY_merged])
D1_merged.to_csv(r'C:/Users/Geraldo Wibowo/Desktop/Year 4/FYP/JUPYTER TEST 1/D1_merged.csv')
print(D1_merged.shape)
print(D1_DROWSY_SECONDARY_merged.shape)

(4799, 39)
(499, 39)


In [74]:
D2_merged = pd.concat([D2_NORMAL1_SECONDARY_merged,
D2_NORMAL2_SECONDARY_merged,
D2_NORMAL_MOTORWAY_merged,
D2_AGGRESSIVE_MOTORWAY_merged,
D2_DROWSY_MOTORWAY_merged,
D2_AGGRESSIVE_SECONDARY_merged,
D2_DROWSY_SECONDARY_merged])
D2_merged.to_csv(r'C:/Users/Geraldo Wibowo/Desktop/Year 4/FYP/JUPYTER TEST 1/D2_merged.csv')
print(D2_merged.shape)

(5267, 39)


In [75]:
D3_merged = pd.concat([D3_NORMAL1_SECONDARY_merged,
D3_NORMAL2_SECONDARY_merged,
D3_NORMAL_MOTORWAY_merged,
D3_AGGRESSIVE_MOTORWAY_merged,
D3_DROWSY_MOTORWAY_merged,
D3_AGGRESSIVE_SECONDARY_merged,
D3_DROWSY_SECONDARY_merged])
D3_merged.to_csv(r'C:/Users/Geraldo Wibowo/Desktop/Year 4/FYP/JUPYTER TEST 1/D3_merged.csv')
print(D3_merged.shape)

(5399, 39)


In [76]:
D4_merged = pd.concat([D4_NORMAL1_SECONDARY_merged,
D4_NORMAL2_SECONDARY_merged,
D4_NORMAL_MOTORWAY_merged,
D4_AGGRESSIVE_MOTORWAY_merged,
D4_DROWSY_MOTORWAY_merged,
D4_AGGRESSIVE_SECONDARY_merged,
D4_DROWSY_SECONDARY_merged])
D4_merged.to_csv(r'C:/Users/Geraldo Wibowo/Desktop/Year 4/FYP/JUPYTER TEST 1/D4_merged.csv')
print(D4_merged.shape)

(5648, 39)


In [77]:
D5_merged = pd.concat([D5_NORMAL1_SECONDARY_merged,
D5_NORMAL2_SECONDARY_merged,
D5_NORMAL_MOTORWAY_merged,
D5_AGGRESSIVE_MOTORWAY_merged,
D5_DROWSY_MOTORWAY_merged,
D5_AGGRESSIVE_SECONDARY_merged,
D5_DROWSY_SECONDARY_merged])
D5_merged.to_csv(r'C:/Users/Geraldo Wibowo/Desktop/Year 4/FYP/JUPYTER TEST 1/D5_merged.csv')
print(D5_merged.shape)

(5292, 39)


In [78]:
D6_merged = pd.concat([D6_NORMAL_SECONDARY_merged,
D6_NORMAL_MOTORWAY_merged,
D6_AGGRESSIVE_MOTORWAY_merged,
D6_DROWSY_MOTORWAY_merged,
D6_DROWSY_SECONDARY_merged])
D6_merged.to_csv(r'C:/Users/Geraldo Wibowo/Desktop/Year 4/FYP/JUPYTER TEST 1/D6_merged.csv')
print(D6_merged.shape)

(4524, 39)
